<a href="https://colab.research.google.com/github/monicasjsu/deep_learning/blob/master/Monica_TitleVsBody_Factor_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## **Title Vs Body**
#### Monica Dommaraju (014545336)
#### Team Name : Shecodes

---

### **Dataset Details**

- Kaggle Fakenews Dataset
- Liar Liar Dataset
- Politifact Fake news and Real News Content
- Stance Dataset from Fake News Challenge


### **Summary**

During my first two sprints, I have been exploring datasets and doing research on how to generate features from title and text, how to amalgamate multiple datasets etc.

I generated couple features releated to similarities between title and text using TF-IDF and Cosine similarities. I have calculated the count of overlapping strong words between title and text. 

Starting third sprint, I started adding more and more features using multiple techinques such as

 - Performed data cleaning on the title and text columns that include Removing symbols, punctuation, stop word removal, Lemmaization, Stemming, etc..
 - Generating n-grams (unigram, bigrams and trigrams)
 - Content statistics that include calculating the lenght of sentences, count of n-grams and count of overlapping n-grams between title and text.
 - I then simplified and enhanced my TD-IDF algorigthm and calculated cosine similaties between the title and text.
 - Used SVD to perform dimensionality reduction on TF-IDF vectors and to perform Latent Semantic Analysis (LSA) as part of topic modelling. 
 - Performed cosine similarities between the title and text topics generated from above step.
 - Used Word2Vec using Google News corpus to find synonyms and next probable words on both title and text and calculated the similarites from the outcome.
 - Performed Sentiment analysis on both title and text was extracted the sentiment scores for (pos, neg, nuetral).
 - Performed normlization on all the distilled features using Standard Scaler. 
 - Ran all the above steps on two datasets. One with multiclass label and the other on the binary classified amalgamated data set.
 - Ran Many different models on both the datasets. (Multi classification models and Binary classification models)
 - Ran Stratified KFold cross validation technique to make sure the model is not overfitting.
 - Analyed the results and derived a polynomical equation to generate the fakeness score by taking the predictions from both multi classification and binary classification models  
 - Randomly picked a few samples and tested them by inputting them to the prediction function which then generated the overall fakeness score.
 - I was able to acheive an overall accuracy of --

### **What did I do and what worked?**
I was able to do all the things that I mentioned above. 

From Sprint 2 to Spring 4, I was able to improve the accuracies of the models from 57% to 88%. I was able to acheive these gains mainly by working on the hybrid model of training models on both Binary and Multi classifaction. I think, the biggest boost was given by the numerous generated features which are very much relevant in identifying the similarities between Title and Text.

### **What did not work?**

Initially, I was trying to amalagamate, Liar-Liar dataset with Kaggle and Politifact datasets. As there was no dedicated title feature in the Liar-Liar dataset, I was using the topic as the title. But that didn't give me any good result. Later I started to explore to find the relevant dataset from Fakenews Challenge, which was apt for this feature as explained above. 

I was also only performing Binary classification at first with only a basic features such as Cosine Similarity between the TF-IDF vectors of title and text. But that gave me poor accuracies. Generating more features from different techniqes worked pretty well.

I realized just working on individual cleaned words is not sufficient. So I worked on unigram bigram and trigram features. 

### **What alternatives did you try?**

I tried to work on Liar-Liar Dataset,but later swithed to Fakenews challenge dataset.

Tried generating multiple polynomial equations and chose the best one after running a few iterations.

I started to perform LDA, but then switched to LSA as I wanted to try how it works, considering that we have already used LDA for other features.

Tried to choose best features by using Gini Index. But then considering the number of features not many, I ended up using all the features by normalizing them using StandardScaler.


# Import Libraries

In [0]:
import pandas as pd
import numpy as np
import csv
import re
import seaborn as sns
from google.colab import drive
import gensim
from scipy.spatial.distance import cosine
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from nltk.stem.porter import PorterStemmer
from sklearn import metrics
from gensim.models.word2vec import Word2Vec
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from string import punctuation
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
import matplotlib.pyplot as plt
from scipy import sparse
from gensim.models.doc2vec import TaggedDocument

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# Load the Datasets

## Kaggle Fake News Dataset

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
fake_train = pd.read_csv("/content/drive/Shared drives/SheCodes/Datasets/fakenews/train_fakenews.csv",sep=',')
fake_test = pd.read_csv("/content/drive/Shared drives/SheCodes/Datasets/fakenews/test_fakenews.csv",sep=',')
fake_submit = pd.read_csv("/content/drive/Shared drives/SheCodes/Datasets/fakenews/submit_fakenews.csv",sep=',')


fake_test = pd.merge(fake_test, fake_submit, on="id", how='left')
fake_train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [0]:
fake_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [0]:
fake_test.head()

,id,title,author,text,label
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...",0
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...,1
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...,0
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different...",1
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,1


In [0]:
fake_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5200 entries, 0 to 5199
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      5200 non-null   int64 
 1   title   5078 non-null   object
 2   author  4697 non-null   object
 3   text    5193 non-null   object
 4   label   5200 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 243.8+ KB


## Liar Liar Dataset

In [0]:
# Load Liar-Liar dataset
columns = [
  'jsonid', 
  'label', 
  'text', 
  'subject', 
  'speaker', 
  'speakerjobtitle', 
  'stateinfo',
  'partyaffiliation', 
  'barelytruecounts', 
  'falsecounts',
  'halftruecounts',
  'mostlytrueocunts',
  'pantsonfirecounts',
  'context'
  ]

train_news = pd.read_csv("/content/drive/Shared drives/SheCodes/Datasets/liar_dataset/train.tsv",sep='\t', names=columns)
test_news = pd.read_csv("/content/drive/Shared drives/SheCodes/Datasets/liar_dataset/test.tsv",sep='\t', names=columns)
valid_news = pd.read_csv("/content/drive/Shared drives/SheCodes/Datasets/liar_dataset/valid.tsv",sep='\t', names=columns)

In [0]:
train_news.head()

,jsonid,label,text,subject,speaker,speakerjobtitle,stateinfo,partyaffiliation,barelytruecounts,falsecounts,halftruecounts,mostlytrueocunts,pantsonfirecounts,context
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN


In [0]:
train_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10240 entries, 0 to 10239
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   jsonid             10240 non-null  object 
 1   label              10240 non-null  object 
 2   text               10240 non-null  object 
 3   subject            10238 non-null  object 
 4   speaker            10238 non-null  object 
 5   speakerjobtitle    7343 non-null   object 
 6   stateinfo          8032 non-null   object 
 7   partyaffiliation   10238 non-null  object 
 8   barelytruecounts   10238 non-null  float64
 9   falsecounts        10238 non-null  float64
 10  halftruecounts     10238 non-null  float64
 11  mostlytrueocunts   10238 non-null  float64
 12  pantsonfirecounts  10238 non-null  float64
 13  context            10138 non-null  object 
dtypes: float64(5), object(9)
memory usage: 1.1+ MB


## Politifact news dataset

In [0]:
politifact_fake = pd.read_csv("/content/drive/Shared drives/SheCodes/Datasets/politifact_dataset/Fake.csv",sep=',')
politifact_fake['label'] = 1
politifact_true = pd.read_csv("/content/drive/Shared drives/SheCodes/Datasets/politifact_dataset/True.csv",sep=',')
politifact_true['label'] = 0
df_politifact = pd.concat([politifact_fake, politifact_true])
df_politifact.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [0]:
df_politifact.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44898 entries, 0 to 21416
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   label    44898 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 2.1+ MB


## Stance Dataset

In [0]:
train_bodies = pd.read_csv("/content/drive/Shared drives/SheCodes/Datasets/stance_dataset/train_bodies.csv")
train_stances = pd.read_csv("/content/drive/Shared drives/SheCodes/Datasets/stance_dataset/train_stances.csv")

In [0]:
test_bodies = pd.read_csv("/content/drive/Shared drives/SheCodes/Datasets/stance_dataset/competition_test_bodies.csv")
test_stances = pd.read_csv("/content/drive/Shared drives/SheCodes/Datasets/stance_dataset/competition_test_stances.csv")

In [0]:
train_stances.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49972 entries, 0 to 49971
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Headline  49972 non-null  object
 1   Body ID   49972 non-null  int64 
 2   Stance    49972 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [0]:
train_bodies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1683 entries, 0 to 1682
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Body ID      1683 non-null   int64 
 1   articleBody  1683 non-null   object
dtypes: int64(1), object(1)
memory usage: 26.4+ KB


In [0]:
test_stances.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25413 entries, 0 to 25412
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Headline  25413 non-null  object
 1   Body ID   25413 non-null  int64 
 2   Stance    25413 non-null  object
dtypes: int64(1), object(2)
memory usage: 595.7+ KB


In [0]:
train_bodies.head()

,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...


In [0]:
train_stances.head()

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,disagree


In [0]:
final_train_stances = pd.merge(left=train_stances, right=train_bodies, how='left', left_on='Body ID', right_on='Body ID')

In [0]:
final_train_stances.head()

,Headline,Body ID,Stance,articleBody
0,Police find mass graves with at least '15 bodi...,712,unrelated,Danny Boyle is directing the untitled film\n\n...
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree,Hundreds of Palestinians were evacuated from t...
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated,30-year-old Moscow resident was hospitalized w...
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated,(Reuters) - A Canadian soldier was shot at the...
4,Spider burrowed through tourist's stomach and ...,1923,disagree,"Fear not arachnophobes, the story of Bunbury's..."


In [0]:
final_train_stances.isna().sum()

Headline       0
Body ID        0
Stance         0
articleBody    0
dtype: int64

In [0]:
final_test_stances = pd.merge(left=test_stances, right=test_bodies, how='left', left_on='Body ID', right_on='Body ID')

In [0]:
final_test_stances.head()

,Headline,Body ID,Stance,articleBody
0,Ferguson riots: Pregnant woman loses eye after...,2008,unrelated,A RESPECTED senior French police officer inves...
1,Crazy Conservatives Are Sure a Gitmo Detainee ...,1550,unrelated,Dave Morin's social networking company Path is...
2,A Russian Guy Says His Justin Bieber Ringtone ...,2,unrelated,A bereaved Afghan mother took revenge on the T...
3,"Zombie Cat: Buried Kitty Believed Dead, Meows ...",1793,unrelated,Hewlett-Packard is officially splitting in two...
4,Argentina's President Adopts Boy to End Werewo...,37,unrelated,An airline passenger headed to Dallas was remo...


In [0]:
final_test_stances.isnull().sum()


Headline       0
Body ID        0
Stance         0
articleBody    0
dtype: int64

In [0]:
final_stance = pd.concat([final_train_stances, final_test_stances], axis=0, ignore_index=True)

In [0]:
final_stance.head()

,Headline,Body ID,Stance,articleBody
0,Police find mass graves with at least '15 bodi...,712,unrelated,Danny Boyle is directing the untitled film\n\n...
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree,Hundreds of Palestinians were evacuated from t...
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated,30-year-old Moscow resident was hospitalized w...
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated,(Reuters) - A Canadian soldier was shot at the...
4,Spider burrowed through tourist's stomach and ...,1923,disagree,"Fear not arachnophobes, the story of Bunbury's..."


In [0]:
final_stance.isna().sum()

Headline       0
Body ID        0
Stance         0
articleBody    0
dtype: int64

In [0]:
final_stance.shape

(75385, 4)

In [0]:
final_stance.drop('Body ID', axis=1, inplace=True)
final_stance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75385 entries, 0 to 75384
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Headline     75385 non-null  object
 1   Stance       75385 non-null  object
 2   articleBody  75385 non-null  object
dtypes: object(3)
memory usage: 1.7+ MB


In [0]:
final_stance.rename(columns={'Headline': 'title', 'articleBody': 'text', 'Stance': 'label'}, inplace=True)


In [0]:
final_stance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75385 entries, 0 to 75384
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   75385 non-null  object
 1   label   75385 non-null  object
 2   text    75385 non-null  object
dtypes: object(3)
memory usage: 1.7+ MB


# Preprocessing

In [0]:
def cleaning(raw_news):
    import nltk
    
    # 1. Remove non-letters/Special Characters and Punctuations
    news = re.sub("[^a-zA-Z]", " ", raw_news)
    
    # 2. Convert to lower case.
    news =  news.lower()
    
    # 3. Tokenize.
    news_words = nltk.word_tokenize( news)
    
    # 4. Convert the stopwords list to "set" data type.
    stops = set(nltk.corpus.stopwords.words("english"))
    
    # 5. Remove stop words. 
    words = [w for w in  news_words  if not w in stops]
    
    # 6. Lemmentize 
    wordnet_lem = [ WordNetLemmatizer().lemmatize(w) for w in words ]
    
    # 7. Stemming
    stems = [nltk.stem.SnowballStemmer('english').stem(w) for w in wordnet_lem ]
    
    # 8. Join the stemmed words back into one string separated by space, and return the result.
    return " ".join(stems)


# Feature 11: Title Vs Body 
**Name: Monica Dommaraju**


My main idea of Title vs Body factor analysis is to pick Title and text features from each of these datasets and try to extract as many distilled features and then run models for classsification.

1. Kaggle Fake News Dataset
2. Liar-Liar Dataset
3. politifact news Dataset
4. Stance Dataset (from FakeNews Detection Challenge)


`kaggle Fake News Dataset` Dataset and `Politifact news dataset` contains the title and text columns, but Liar-Liar Dataset contains only text column with out title text (I beleive `subject` feature in the Liar-Liar Dataset is not same the title). This made look for alternative datasets, so I can work on multi label classification. 

  I found Stance Dataset posted by FakeNews Detection


and perform the following steps
* Amalgamate all three datasets
* cleaning the dataset 
* visualizing it using wordcloud
* Calculate cosine similarity between Tile and Body for each sample
* Calculate Number of matching strong words between these two columns
* Run multiple classification models

## Amalgamation
**Amalgamate Kaggle Fake News Dataset and Politifact News Dataset**

In [0]:
df_kaggle = pd.concat([fake_train, fake_test])
df_kaggle.shape
# Pick only title and text columns from Kaggle dataset
df_final = df_kaggle[['title', 'text', 'label']]
df_final.shape

(26000, 3)

In [0]:
df_final = df_final.append(df_politifact[['title', 'text', 'label']])

# Todo: Remove this sample technique when running the model finally
# Running it with 25% of sample due to hardware constraints
df_final = df_final.sample(frac=0.01).reset_index(drop=True)
print (df_final.shape)
df_final.head()

(709, 3)


,title,text,label
0,Indian tycoon's extradition hearing told of ab...,LONDON (Reuters) - One of the Chennai Six gr...,0
1,Iran Proves Republicans Are Full Of Crap As U...,Iran is fulfilling their obligations under the...,1
2,Breaking: Dems Keep Progressive Bent In All-I...,Nancy Pelosi has been the leader of the House ...,1
3,NaN,Watch this video. Fully documented the heavy c...,1
4,Centrum – The World's Most Popular Multivitami...,"Share on Facebook This can result in fatigue, ...",1


In [0]:
# Todo: Remove this sample technique when running the model finally
# Running it with 25% of sample due to hardware constraint

final_stance = final_stance.sample(frac=0.01).reset_index(drop=True)
print(final_stance.shape)
final_stance.head()

(754, 3)


,title,label,text
0,Has Ebola infected Isis militants in Mosul?,discuss,The World Health Organisation is investigating...
1,Meteor Leaves 40-Foot Crater Near Managua's Ai...,unrelated,ISIS released a video purportedly showing the ...
2,Mexico prosecutor: Students not in 1st mass gr...,unrelated,After being officially dead for 48 minutes and...
3,Letter From Lego To Parents In The '70s Makes ...,unrelated,HBO's subscription streaming service will be c...
4,Islamic State Militants Have Been Flying 3 Cap...,discuss,By Sylvia Westall\n\nBEIRUT (Reuters) – Iraqi ...


## Clean the title and text

In [0]:
# Cleaning on Fake News Dataset
final_stance['cleaned_title'] = final_stance["title"].map(lambda x: cleaning(x))
final_stance['cleaned_text'] = final_stance["text"].map(lambda x: cleaning(x))
final_stance.head()

,title,label,text,cleaned_title,cleaned_text
0,Has Ebola infected Isis militants in Mosul?,discuss,The World Health Organisation is investigating...,ebola infect isi milit mosul,world health organis investig report isi milit...
1,Meteor Leaves 40-Foot Crater Near Managua's Ai...,unrelated,ISIS released a video purportedly showing the ...,meteor leaf foot crater near managua airport,isi releas video purport show execut british a...
2,Mexico prosecutor: Students not in 1st mass gr...,unrelated,After being officially dead for 48 minutes and...,mexico prosecutor student st mass graf,offici dead minut heart restart year old massa...
3,Letter From Lego To Parents In The '70s Makes ...,unrelated,HBO's subscription streaming service will be c...,letter lego parent make import point gender,hbo subscript stream servic call hbo expect co...
4,Islamic State Militants Have Been Flying 3 Cap...,discuss,By Sylvia Westall\n\nBEIRUT (Reuters) – Iraqi ...,islam state milit fli captur warplan syrian ai...,sylvia westal beirut reuter iraqi pilot join i...


In [0]:
# Cleaning on Amalamated Kaggle and Fake News dataset.
print (final_stance.info())
print (df_final.info())
df_final['title'] = df_final['title'].astype('str')
df_final['text'] = df_final['text'].astype('str')
df_final['cleaned_title'] = df_final['title'].astype('str').map(lambda x: cleaning(x))
df_final['cleaned_text'] = df_final['text'].astype('str').map(lambda x: cleaning(x))
df_final.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 754 entries, 0 to 753
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          754 non-null    object
 1   label          754 non-null    object
 2   text           754 non-null    object
 3   cleaned_title  754 non-null    object
 4   cleaned_text   754 non-null    object
dtypes: object(5)
memory usage: 29.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709 entries, 0 to 708
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   700 non-null    object
 1   text    708 non-null    object
 2   label   709 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 16.7+ KB
None


,title,text,label,cleaned_title,cleaned_text
0,Indian tycoon's extradition hearing told of ab...,LONDON (Reuters) - One of the Chennai Six gr...,0,indian tycoon extradit hear told abus uk chenn...,london reuter one chennai six group ex british...
1,Iran Proves Republicans Are Full Of Crap As U...,Iran is fulfilling their obligations under the...,1,iran prof republican full crap unit nation say...,iran fulfil oblig nuclear deal reach obama adm...
2,Breaking: Dems Keep Progressive Bent In All-I...,Nancy Pelosi has been the leader of the House ...,1,break dem keep progress bent import leadership...,nanci pelosi leader hous democrat long time we...
3,nan,Watch this video. Fully documented the heavy c...,1,nan,watch video fulli document heavi connect islam...
4,Centrum – The World's Most Popular Multivitami...,"Share on Facebook This can result in fatigue, ...",1,centrum world popular multivitamin useless con...,share facebook result fatigu dizzi short breat...


##Count of sentences in Title and Text 

In [0]:
from nltk.tokenize import sent_tokenize

def count_sentences(data):
  data['count_title_sentences'] = data['title'].apply(lambda x: len(sent_tokenize(x)))
  data['count_text_sentences'] = data['text'].apply(lambda x: len(sent_tokenize(x)))

In [0]:
# Calculate the total number of sentences in Title and Text on Multiclass labeled Dataset
count_sentences(final_stance)
final_stance.head()

,title,label,text,cleaned_title,cleaned_text,count_title_sentences,count_text_sentences
0,Has Ebola infected Isis militants in Mosul?,discuss,The World Health Organisation is investigating...,ebola infect isi milit mosul,world health organis investig report isi milit...,1,18
1,Meteor Leaves 40-Foot Crater Near Managua's Ai...,unrelated,ISIS released a video purportedly showing the ...,meteor leaf foot crater near managua airport,isi releas video purport show execut british a...,1,1
2,Mexico prosecutor: Students not in 1st mass gr...,unrelated,After being officially dead for 48 minutes and...,mexico prosecutor student st mass graf,offici dead minut heart restart year old massa...,1,9
3,Letter From Lego To Parents In The '70s Makes ...,unrelated,HBO's subscription streaming service will be c...,letter lego parent make import point gender,hbo subscript stream servic call hbo expect co...,1,16
4,Islamic State Militants Have Been Flying 3 Cap...,discuss,By Sylvia Westall\n\nBEIRUT (Reuters) – Iraqi ...,islam state milit fli captur warplan syrian ai...,sylvia westal beirut reuter iraqi pilot join i...,1,18


In [0]:
# Calculate the total number of sentences in Title and Text on Binary class labeled Dataset
count_sentences(df_final)
df_final.head()

,title,text,label,cleaned_title,cleaned_text,count_title_sentences,count_text_sentences
0,Indian tycoon's extradition hearing told of ab...,LONDON (Reuters) - One of the Chennai Six gr...,0,indian tycoon extradit hear told abus uk chenn...,london reuter one chennai six group ex british...,1,13
1,Iran Proves Republicans Are Full Of Crap As U...,Iran is fulfilling their obligations under the...,1,iran prof republican full crap unit nation say...,iran fulfil oblig nuclear deal reach obama adm...,1,20
2,Breaking: Dems Keep Progressive Bent In All-I...,Nancy Pelosi has been the leader of the House ...,1,break dem keep progress bent import leadership...,nanci pelosi leader hous democrat long time we...,1,17
3,nan,Watch this video. Fully documented the heavy c...,1,nan,watch video fulli document heavi connect islam...,1,5
4,Centrum – The World's Most Popular Multivitami...,"Share on Facebook This can result in fatigue, ...",1,centrum world popular multivitamin useless con...,share facebook result fatigu dizzi short breat...,1,50


## Generating N-Grams and Their Lengths

Uni, Bi and Tri grams are generated so we can extract some count fearures for each of these grams along with the count features of sentences.

In [0]:
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

def ngram(text, n):
    n_grams = ngrams(word_tokenize(text), n)
    return [ '_'.join(grams) for grams in n_grams]

In [0]:
def generate_ngram_features(data):
  data["title_unigram"] = data["cleaned_title"].map(lambda x: ngram(x, 1))
  data["text_unigram"] = data["cleaned_text"].map(lambda x: ngram(x, 1))
  data["count_title_unigram"] = list(data.apply(lambda x: len(x['title_unigram']), axis=1))
  data["count_text_unigram"] = list(data.apply(lambda x: len(x['text_unigram']), axis=1))
  data["unique_count_title_unigram"] = list(data.apply(lambda x: len(set(x['title_unigram'])), axis=1))
  data["unique_count_text_unigram"] = list(data.apply(lambda x: len(set(x['text_unigram'])), axis=1))

  data["title_bigram"] = data["cleaned_title"].map(lambda x: ngram(x, 2))
  data["text_bigram"] = data["cleaned_text"].map(lambda x: ngram(x, 2))
  data["count_title_bigram"] = list(data.apply(lambda x: len(x['title_bigram']), axis=1))
  data["count_text_bigram"] = list(data.apply(lambda x: len(x['text_bigram']), axis=1))
  data["unique_count_title_bigram"] = list(data.apply(lambda x: len(set(x['title_bigram'])), axis=1))
  data["unique_count_text_bigram"] = list(data.apply(lambda x: len(set(x['text_bigram'])), axis=1))

  data["title_trigram"] = data["cleaned_title"].map(lambda x: ngram(x, 3))
  data["text_trigram"] = data["cleaned_text"].map(lambda x: ngram(x, 3))
  data["count_title_trigram"] = list(data.apply(lambda x: len(x['title_trigram']), axis=1))
  data["count_text_trigram"] = list(data.apply(lambda x: len(x['text_trigram']), axis=1))
  data["unique_count_title_trigram"] = list(data.apply(lambda x: len(set(x['title_trigram'])), axis=1))
  data["unique_count_text_trigram"] = list(data.apply(lambda x: len(set(x['text_trigram'])), axis=1))

In [0]:
# Generate n-grams of title and text. Then calculate the length of unigram, bigram and trigrams on Multiclass labeled DS
generate_ngram_features(final_stance)
final_stance.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: generator 'ngrams' raised StopIteration
  


,title,label,text,cleaned_title,cleaned_text,count_title_sentences,count_text_sentences,title_unigram,text_unigram,count_title_unigram,count_text_unigram,unique_count_title_unigram,unique_count_text_unigram,title_bigram,text_bigram,count_title_bigram,count_text_bigram,unique_count_title_bigram,unique_count_text_bigram,title_trigram,text_trigram,count_title_trigram,count_text_trigram,unique_count_title_trigram,unique_count_text_trigram
0,Has Ebola infected Isis militants in Mosul?,discuss,The World Health Organisation is investigating...,ebola infect isi milit mosul,world health organis investig report isi milit...,1,18,"[ebola, infect, isi, milit, mosul]","[world, health, organis, investig, report, isi...",5,260,5,160,"[ebola_infect, infect_isi, isi_milit, milit_mo...","[world_health, health_organis, organis_investi...",4,259,4,231,"[ebola_infect_isi, infect_isi_milit, isi_milit...","[world_health_organis, health_organis_investig...",3,258,3,240
1,Meteor Leaves 40-Foot Crater Near Managua's Ai...,unrelated,ISIS released a video purportedly showing the ...,meteor leaf foot crater near managua airport,isi releas video purport show execut british a...,1,1,"[meteor, leaf, foot, crater, near, managua, ai...","[isi, releas, video, purport, show, execut, br...",7,11,7,11,"[meteor_leaf, leaf_foot, foot_crater, crater_n...","[isi_releas, releas_video, video_purport, purp...",6,10,6,10,"[meteor_leaf_foot, leaf_foot_crater, foot_crat...","[isi_releas_video, releas_video_purport, video...",5,9,5,9
2,Mexico prosecutor: Students not in 1st mass gr...,unrelated,After being officially dead for 48 minutes and...,mexico prosecutor student st mass graf,offici dead minut heart restart year old massa...,1,9,"[mexico, prosecutor, student, st, mass, graf]","[offici, dead, minut, heart, restart, year, ol...",6,78,6,69,"[mexico_prosecutor, prosecutor_student, studen...","[offici_dead, dead_minut, minut_heart, heart_r...",5,77,5,77,"[mexico_prosecutor_student, prosecutor_student...","[offici_dead_minut, dead_minut_heart, minut_he...",4,76,4,76
3,Letter From Lego To Parents In The '70s Makes ...,unrelated,HBO's subscription streaming service will be c...,letter lego parent make import point gender,hbo subscript stream servic call hbo expect co...,1,16,"[letter, lego, parent, make, import, point, ge...","[hbo, subscript, stream, servic, call, hbo, ex...",7,229,7,137,"[letter_lego, lego_parent, parent_make, make_i...","[hbo_subscript, subscript_stream, stream_servi...",6,228,6,212,"[letter_lego_parent, lego_parent_make, parent_...","[hbo_subscript_stream, subscript_stream_servic...",5,227,5,223
4,Islamic State Militants Have Been Flying 3 Cap...,discuss,By Sylvia Westall\n\nBEIRUT (Reuters) – Iraqi ...,islam state milit fli captur warplan syrian ai...,sylvia westal beirut reuter iraqi pilot join i...,1,18,"[islam, state, milit, fli, captur, warplan, sy...","[sylvia, westal, beirut, reuter, iraqi, pilot,...",10,324,10,191,"[islam_state, state_milit, milit_fli, fli_capt...","[sylvia_westal, westal_beirut, beirut_reuter, ...",9,323,9,299,"[islam_state_milit, state_milit_fli, milit_fli...","[sylvia_westal_beirut, westal_beirut_reuter, b...",8,322,8,315


In [0]:
# Generate n-grams of title and text. Then calculate the length of unigram, bigram and trigrams on Binary class labeled DS
generate_ngram_features(df_final)
df_final.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: generator 'ngrams' raised StopIteration
  


,title,text,label,cleaned_title,cleaned_text,count_title_sentences,count_text_sentences,title_unigram,text_unigram,count_title_unigram,count_text_unigram,unique_count_title_unigram,unique_count_text_unigram,title_bigram,text_bigram,count_title_bigram,count_text_bigram,unique_count_title_bigram,unique_count_text_bigram,title_trigram,text_trigram,count_title_trigram,count_text_trigram,unique_count_title_trigram,unique_count_text_trigram
0,Indian tycoon's extradition hearing told of ab...,LONDON (Reuters) - One of the Chennai Six gr...,0,indian tycoon extradit hear told abus uk chenn...,london reuter one chennai six group ex british...,1,13,"[indian, tycoon, extradit, hear, told, abus, u...","[london, reuter, one, chennai, six, group, ex,...",9,251,9,162,"[indian_tycoon, tycoon_extradit, extradit_hear...","[london_reuter, reuter_one, one_chennai, chenn...",8,250,8,238,"[indian_tycoon_extradit, tycoon_extradit_hear,...","[london_reuter_one, reuter_one_chennai, one_ch...",7,249,7,246
1,Iran Proves Republicans Are Full Of Crap As U...,Iran is fulfilling their obligations under the...,1,iran prof republican full crap unit nation say...,iran fulfil oblig nuclear deal reach obama adm...,1,20,"[iran, prof, republican, full, crap, unit, nat...","[iran, fulfil, oblig, nuclear, deal, reach, ob...",11,422,11,266,"[iran_prof, prof_republican, republican_full, ...","[iran_fulfil, fulfil_oblig, oblig_nuclear, nuc...",10,421,10,401,"[iran_prof_republican, prof_republican_full, r...","[iran_fulfil_oblig, fulfil_oblig_nuclear, obli...",9,420,9,418
2,Breaking: Dems Keep Progressive Bent In All-I...,Nancy Pelosi has been the leader of the House ...,1,break dem keep progress bent import leadership...,nanci pelosi leader hous democrat long time we...,1,17,"[break, dem, keep, progress, bent, import, lea...","[nanci, pelosi, leader, hous, democrat, long, ...",9,225,9,165,"[break_dem, dem_keep, keep_progress, progress_...","[nanci_pelosi, pelosi_leader, leader_hous, hou...",8,224,8,214,"[break_dem_keep, dem_keep_progress, keep_progr...","[nanci_pelosi_leader, pelosi_leader_hous, lead...",7,223,7,222
3,nan,Watch this video. Fully documented the heavy c...,1,nan,watch video fulli document heavi connect islam...,1,5,[nan],"[watch, video, fulli, document, heavi, connect...",1,30,1,25,[],"[watch_video, video_fulli, fulli_document, doc...",0,29,0,28,[],"[watch_video_fulli, video_fulli_document, full...",0,28,0,28
4,Centrum – The World's Most Popular Multivitami...,"Share on Facebook This can result in fatigue, ...",1,centrum world popular multivitamin useless con...,share facebook result fatigu dizzi short breat...,1,50,"[centrum, world, popular, multivitamin, useles...","[share, facebook, result, fatigu, dizzi, short...",10,434,10,306,"[centrum_world, world_popular, popular_multivi...","[share_facebook, facebook_result, result_fatig...",9,433,9,426,"[centrum_world_popular, world_popular_multivit...","[share_facebook_result, facebook_result_fatigu...",8,432,8,432


## Common N-grams between Title and Text

In [0]:
def common_ngrams_in_text(data):
  data["count_title_unigrams_in_text"] =  list(data.apply(lambda x: sum([1. for w in x['title_unigram'] if w in set(x['text_unigram'])]), axis=1))
  data["count_title_bigrams_in_text"] =  list(data.apply(lambda x: sum([1. for w in x['title_bigram'] if w in set(x['text_bigram'])]), axis=1))
  data["count_title_trigrams_in_text"] =  list(data.apply(lambda x: sum([1. for w in x['title_trigram'] if w in set(x['text_trigram'])]), axis=1))

In [0]:
# Calculate the total number of common title n-grams in the text on Multi class labeled DS
common_ngrams_in_text(final_stance)
final_stance.head()

,title,label,text,cleaned_title,cleaned_text,count_title_sentences,count_text_sentences,title_unigram,text_unigram,count_title_unigram,count_text_unigram,unique_count_title_unigram,unique_count_text_unigram,title_bigram,text_bigram,count_title_bigram,count_text_bigram,unique_count_title_bigram,unique_count_text_bigram,title_trigram,text_trigram,count_title_trigram,count_text_trigram,unique_count_title_trigram,unique_count_text_trigram,count_title_unigrams_in_text,count_title_bigrams_in_text,count_title_trigrams_in_text
0,Has Ebola infected Isis militants in Mosul?,discuss,The World Health Organisation is investigating...,ebola infect isi milit mosul,world health organis investig report isi milit...,1,18,"[ebola, infect, isi, milit, mosul]","[world, health, organis, investig, report, isi...",5,260,5,160,"[ebola_infect, infect_isi, isi_milit, milit_mo...","[world_health, health_organis, organis_investi...",4,259,4,231,"[ebola_infect_isi, infect_isi_milit, isi_milit...","[world_health_organis, health_organis_investig...",3,258,3,240,4.0,1.0,0.0
1,Meteor Leaves 40-Foot Crater Near Managua's Ai...,unrelated,ISIS released a video purportedly showing the ...,meteor leaf foot crater near managua airport,isi releas video purport show execut british a...,1,1,"[meteor, leaf, foot, crater, near, managua, ai...","[isi, releas, video, purport, show, execut, br...",7,11,7,11,"[meteor_leaf, leaf_foot, foot_crater, crater_n...","[isi_releas, releas_video, video_purport, purp...",6,10,6,10,"[meteor_leaf_foot, leaf_foot_crater, foot_crat...","[isi_releas_video, releas_video_purport, video...",5,9,5,9,0.0,0.0,0.0
2,Mexico prosecutor: Students not in 1st mass gr...,unrelated,After being officially dead for 48 minutes and...,mexico prosecutor student st mass graf,offici dead minut heart restart year old massa...,1,9,"[mexico, prosecutor, student, st, mass, graf]","[offici, dead, minut, heart, restart, year, ol...",6,78,6,69,"[mexico_prosecutor, prosecutor_student, studen...","[offici_dead, dead_minut, minut_heart, heart_r...",5,77,5,77,"[mexico_prosecutor_student, prosecutor_student...","[offici_dead_minut, dead_minut_heart, minut_he...",4,76,4,76,0.0,0.0,0.0
3,Letter From Lego To Parents In The '70s Makes ...,unrelated,HBO's subscription streaming service will be c...,letter lego parent make import point gender,hbo subscript stream servic call hbo expect co...,1,16,"[letter, lego, parent, make, import, point, ge...","[hbo, subscript, stream, servic, call, hbo, ex...",7,229,7,137,"[letter_lego, lego_parent, parent_make, make_i...","[hbo_subscript, subscript_stream, stream_servi...",6,228,6,212,"[letter_lego_parent, lego_parent_make, parent_...","[hbo_subscript_stream, subscript_stream_servic...",5,227,5,223,2.0,0.0,0.0
4,Islamic State Militants Have Been Flying 3 Cap...,discuss,By Sylvia Westall\n\nBEIRUT (Reuters) – Iraqi ...,islam state milit fli captur warplan syrian ai...,sylvia westal beirut reuter iraqi pilot join i...,1,18,"[islam, state, milit, fli, captur, warplan, sy...","[sylvia, westal, beirut, reuter, iraqi, pilot,...",10,324,10,191,"[islam_state, state_milit, milit_fli, fli_capt...","[sylvia_westal, westal_beirut, beirut_reuter, ...",9,323,9,299,"[islam_state_milit, state_milit_fli, milit_fli...","[sylvia_westal_beirut, westal_beirut_reuter, b...",8,322,8,315,10.0,1.0,0.0


In [0]:
# Calculate the total number of common title n-grams in the text on Binary class labeled DS
common_ngrams_in_text(df_final)
df_final.head()

,title,text,label,cleaned_title,cleaned_text,count_title_sentences,count_text_sentences,title_unigram,text_unigram,count_title_unigram,count_text_unigram,unique_count_title_unigram,unique_count_text_unigram,title_bigram,text_bigram,count_title_bigram,count_text_bigram,unique_count_title_bigram,unique_count_text_bigram,title_trigram,text_trigram,count_title_trigram,count_text_trigram,unique_count_title_trigram,unique_count_text_trigram,count_title_unigrams_in_text,count_title_bigrams_in_text,count_title_trigrams_in_text
0,Indian tycoon's extradition hearing told of ab...,LONDON (Reuters) - One of the Chennai Six gr...,0,indian tycoon extradit hear told abus uk chenn...,london reuter one chennai six group ex british...,1,13,"[indian, tycoon, extradit, hear, told, abus, u...","[london, reuter, one, chennai, six, group, ex,...",9,251,9,162,"[indian_tycoon, tycoon_extradit, extradit_hear...","[london_reuter, reuter_one, one_chennai, chenn...",8,250,8,238,"[indian_tycoon_extradit, tycoon_extradit_hear,...","[london_reuter_one, reuter_one_chennai, one_ch...",7,249,7,246,8.0,2.0,0.0
1,Iran Proves Republicans Are Full Of Crap As U...,Iran is fulfilling their obligations under the...,1,iran prof republican full crap unit nation say...,iran fulfil oblig nuclear deal reach obama adm...,1,20,"[iran, prof, republican, full, crap, unit, nat...","[iran, fulfil, oblig, nuclear, deal, reach, ob...",11,422,11,266,"[iran_prof, prof_republican, republican_full, ...","[iran_fulfil, fulfil_oblig, oblig_nuclear, nuc...",10,421,10,401,"[iran_prof_republican, prof_republican_full, r...","[iran_fulfil_oblig, fulfil_oblig_nuclear, obli...",9,420,9,418,8.0,3.0,0.0
2,Breaking: Dems Keep Progressive Bent In All-I...,Nancy Pelosi has been the leader of the House ...,1,break dem keep progress bent import leadership...,nanci pelosi leader hous democrat long time we...,1,17,"[break, dem, keep, progress, bent, import, lea...","[nanci, pelosi, leader, hous, democrat, long, ...",9,225,9,165,"[break_dem, dem_keep, keep_progress, progress_...","[nanci_pelosi, pelosi_leader, leader_hous, hou...",8,224,8,214,"[break_dem_keep, dem_keep_progress, keep_progr...","[nanci_pelosi_leader, pelosi_leader_hous, lead...",7,223,7,222,4.0,0.0,0.0
3,nan,Watch this video. Fully documented the heavy c...,1,nan,watch video fulli document heavi connect islam...,1,5,[nan],"[watch, video, fulli, document, heavi, connect...",1,30,1,25,[],"[watch_video, video_fulli, fulli_document, doc...",0,29,0,28,[],"[watch_video_fulli, video_fulli_document, full...",0,28,0,28,0.0,0.0,0.0
4,Centrum – The World's Most Popular Multivitami...,"Share on Facebook This can result in fatigue, ...",1,centrum world popular multivitamin useless con...,share facebook result fatigu dizzi short breat...,1,50,"[centrum, world, popular, multivitamin, useles...","[share, facebook, result, fatigu, dizzi, short...",10,434,10,306,"[centrum_world, world_popular, popular_multivi...","[share_facebook, facebook_result, result_fatig...",9,433,9,426,"[centrum_world_popular, world_popular_multivit...","[share_facebook_result, facebook_result_fatigu...",8,432,8,432,9.0,1.0,0.0


## TF_IDF and Cosine Similarities

In [0]:
def concat_title_text(data):
  data['cleaned_title_text'] = data['cleaned_title'] + ' ' + data['cleaned_text']

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tf_idf(data):
  concat_title_text(data)
  combined_vectors = TfidfVectorizer(ngram_range=(1, 2), min_df=1, max_df=1, use_idf=True, smooth_idf=True)
  combined_vectors.fit(data["cleaned_title_text"])
  combined_vectors_dictionary = combined_vectors.vocabulary_
  # print (combined_vectors_dictionary)

  # Now lets generate the TfIdf Vectors seperately for Title and Text from the vector dictionary
  # built above
  title_vectors = TfidfVectorizer(ngram_range=(1, 2), min_df=1, max_df=1, use_idf=True, smooth_idf=True, vocabulary=combined_vectors_dictionary)
  title_tfidf_vectors = title_vectors.fit_transform(data['cleaned_title'])
  print (title_tfidf_vectors.shape)
  # print ("\n Title TF-IDF Vectors")
  # print (title_vectors.get_feature_names())     

  text_vectors = TfidfVectorizer(ngram_range=(1, 2), min_df=1, max_df=1, use_idf=True, smooth_idf=True, vocabulary=combined_vectors_dictionary)
  text_tfidf_vectors = text_vectors.fit_transform(data['cleaned_text'])

  # print ("\n Text TF-IDF Vectors")
  # print (text_vectors.get_feature_names())
  print (text_tfidf_vectors.shape)

  return title_tfidf_vectors, text_tfidf_vectors

In [0]:
from scipy.spatial.distance import cosine

def similarity_score(data, title_vectors, text_vectors):
  similarity_score = []
  for i in range(len(data)):
      similarity_score.append(1 - cosine(title_vectors[i], text_vectors[i]))
  return similarity_score

def tf_idf_similarity_score(data):
  title_tfidf_vectors, text_tfidf_vectors = tf_idf(data)
  data['similarity_title_text'] = similarity_score(data, title_tfidf_vectors.toarray(), text_tfidf_vectors.toarray())
  return title_tfidf_vectors, text_tfidf_vectors

In [0]:
# Tf-IDF and Cosine Similarities on Multiclass labeled dataset
stance_title_tfidf_vectors, stance_text_tfidf_vectors = tf_idf_similarity_score(final_stance)
final_stance.head()

(754, 61079)
(754, 61079)


/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


,title,label,text,cleaned_title,cleaned_text,count_title_sentences,count_text_sentences,title_unigram,text_unigram,count_title_unigram,count_text_unigram,unique_count_title_unigram,unique_count_text_unigram,title_bigram,text_bigram,count_title_bigram,count_text_bigram,unique_count_title_bigram,unique_count_text_bigram,title_trigram,text_trigram,count_title_trigram,count_text_trigram,unique_count_title_trigram,unique_count_text_trigram,count_title_unigrams_in_text,count_title_bigrams_in_text,count_title_trigrams_in_text,cleaned_title_text,similarity_title_text
0,Has Ebola infected Isis militants in Mosul?,discuss,The World Health Organisation is investigating...,ebola infect isi milit mosul,world health organis investig report isi milit...,1,18,"[ebola, infect, isi, milit, mosul]","[world, health, organis, investig, report, isi...",5,260,5,160,"[ebola_infect, infect_isi, isi_milit, milit_mo...","[world_health, health_organis, organis_investi...",4,259,4,231,"[ebola_infect_isi, infect_isi_milit, isi_milit...","[world_health_organis, health_organis_investig...",3,258,3,240,4.0,1.0,0.0,ebola infect isi milit mosul world health orga...,NaN
1,Meteor Leaves 40-Foot Crater Near Managua's Ai...,unrelated,ISIS released a video purportedly showing the ...,meteor leaf foot crater near managua airport,isi releas video purport show execut british a...,1,1,"[meteor, leaf, foot, crater, near, managua, ai...","[isi, releas, video, purport, show, execut, br...",7,11,7,11,"[meteor_leaf, leaf_foot, foot_crater, crater_n...","[isi_releas, releas_video, video_purport, purp...",6,10,6,10,"[meteor_leaf_foot, leaf_foot_crater, foot_crat...","[isi_releas_video, releas_video_purport, video...",5,9,5,9,0.0,0.0,0.0,meteor leaf foot crater near managua airport i...,NaN
2,Mexico prosecutor: Students not in 1st mass gr...,unrelated,After being officially dead for 48 minutes and...,mexico prosecutor student st mass graf,offici dead minut heart restart year old massa...,1,9,"[mexico, prosecutor, student, st, mass, graf]","[offici, dead, minut, heart, restart, year, ol...",6,78,6,69,"[mexico_prosecutor, prosecutor_student, studen...","[offici_dead, dead_minut, minut_heart, heart_r...",5,77,5,77,"[mexico_prosecutor_student, prosecutor_student...","[offici_dead_minut, dead_minut_heart, minut_he...",4,76,4,76,0.0,0.0,0.0,mexico prosecutor student st mass graf offici ...,NaN
3,Letter From Lego To Parents In The '70s Makes ...,unrelated,HBO's subscription streaming service will be c...,letter lego parent make import point gender,hbo subscript stream servic call hbo expect co...,1,16,"[letter, lego, parent, make, import, point, ge...","[hbo, subscript, stream, servic, call, hbo, ex...",7,229,7,137,"[letter_lego, lego_parent, parent_make, make_i...","[hbo_subscript, subscript_stream, stream_servi...",6,228,6,212,"[letter_lego_parent, lego_parent_make, parent_...","[hbo_subscript_stream, subscript_stream_servic...",5,227,5,223,2.0,0.0,0.0,letter lego parent make import point gender hb...,0.0
4,Islamic State Militants Have Been Flying 3 Cap...,discuss,By Sylvia Westall\n\nBEIRUT (Reuters) – Iraqi ...,islam state milit fli captur warplan syrian ai...,sylvia westal beirut reuter iraqi pilot join i...,1,18,"[islam, state, milit, fli, captur, warplan, sy...","[sylvia, westal, beirut, reuter, iraqi, pilot,...",10,324,10,191,"[islam_state, state_milit, milit_fli, fli_capt...","[sylvia_westal, westal_beirut, beirut_reuter, ...",9,323,9,299,"[islam_state_milit, state_milit_fli, milit_fli...","[sylvia_westal_beirut, westal_beirut_reuter, b...",8,322,8,315,10.0,1.0,0.0,islam state milit fli captur warplan syrian ai...,NaN


In [0]:
# Tf-IDF and Cosine Similarities on Binary class labeled dataset

df_title_tfidf_vectors, df_text_tfidf_vectors = tf_idf_similarity_score(df_final)
df_final.head()

(709, 164516)
(709, 164516)


/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


,title,text,label,cleaned_title,cleaned_text,count_title_sentences,count_text_sentences,title_unigram,text_unigram,count_title_unigram,count_text_unigram,unique_count_title_unigram,unique_count_text_unigram,title_bigram,text_bigram,count_title_bigram,count_text_bigram,unique_count_title_bigram,unique_count_text_bigram,title_trigram,text_trigram,count_title_trigram,count_text_trigram,unique_count_title_trigram,unique_count_text_trigram,count_title_unigrams_in_text,count_title_bigrams_in_text,count_title_trigrams_in_text,cleaned_title_text,similarity_title_text
0,Indian tycoon's extradition hearing told of ab...,LONDON (Reuters) - One of the Chennai Six gr...,0,indian tycoon extradit hear told abus uk chenn...,london reuter one chennai six group ex british...,1,13,"[indian, tycoon, extradit, hear, told, abus, u...","[london, reuter, one, chennai, six, group, ex,...",9,251,9,162,"[indian_tycoon, tycoon_extradit, extradit_hear...","[london_reuter, reuter_one, one_chennai, chenn...",8,250,8,238,"[indian_tycoon_extradit, tycoon_extradit_hear,...","[london_reuter_one, reuter_one_chennai, one_ch...",7,249,7,246,8.0,2.0,0.0,indian tycoon extradit hear told abus uk chenn...,0.134715
1,Iran Proves Republicans Are Full Of Crap As U...,Iran is fulfilling their obligations under the...,1,iran prof republican full crap unit nation say...,iran fulfil oblig nuclear deal reach obama adm...,1,20,"[iran, prof, republican, full, crap, unit, nat...","[iran, fulfil, oblig, nuclear, deal, reach, ob...",11,422,11,266,"[iran_prof, prof_republican, republican_full, ...","[iran_fulfil, fulfil_oblig, oblig_nuclear, nuc...",10,421,10,401,"[iran_prof_republican, prof_republican_full, r...","[iran_fulfil_oblig, fulfil_oblig_nuclear, obli...",9,420,9,418,8.0,3.0,0.0,iran prof republican full crap unit nation say...,0.019066
2,Breaking: Dems Keep Progressive Bent In All-I...,Nancy Pelosi has been the leader of the House ...,1,break dem keep progress bent import leadership...,nanci pelosi leader hous democrat long time we...,1,17,"[break, dem, keep, progress, bent, import, lea...","[nanci, pelosi, leader, hous, democrat, long, ...",9,225,9,165,"[break_dem, dem_keep, keep_progress, progress_...","[nanci_pelosi, pelosi_leader, leader_hous, hou...",8,224,8,214,"[break_dem_keep, dem_keep_progress, keep_progr...","[nanci_pelosi_leader, pelosi_leader_hous, lead...",7,223,7,222,4.0,0.0,0.0,break dem keep progress bent import leadership...,0.000000
3,nan,Watch this video. Fully documented the heavy c...,1,nan,watch video fulli document heavi connect islam...,1,5,[nan],"[watch, video, fulli, document, heavi, connect...",1,30,1,25,[],"[watch_video, video_fulli, fulli_document, doc...",0,29,0,28,[],"[watch_video_fulli, video_fulli_document, full...",0,28,0,28,0.0,0.0,0.0,nan watch video fulli document heavi connect i...,NaN
4,Centrum – The World's Most Popular Multivitami...,"Share on Facebook This can result in fatigue, ...",1,centrum world popular multivitamin useless con...,share facebook result fatigu dizzi short breat...,1,50,"[centrum, world, popular, multivitamin, useles...","[share, facebook, result, fatigu, dizzi, short...",10,434,10,306,"[centrum_world, world_popular, popular_multivi...","[share_facebook, facebook_result, result_fatig...",9,433,9,426,"[centrum_world_popular, world_popular_multivit...","[share_facebook_result, facebook_result_fatigu...",8,432,8,432,9.0,1.0,0.0,centrum world popular multivitamin useless con...,0.224354


## Word2Vec using Google Corpus and Similarity scores

Now we will generate the word vectors for title and text by training each of the unigram words on Google News corpus. This is used to find synonyms for a word, which will then be used to calculate the similaries scores between the synonyms rather than the exact words.

To use Word2Vec, we need title and text with out lemmatization and stop words performed on it.


In [0]:
def cleaning_simple_tokenize(raw_news):
    import nltk
    
    # 1. Remove non-letters/Special Characters and Punctuations
    news = re.sub("[^a-zA-Z]", " ", raw_news)
    
    # 2. Convert to lower case.
    news =  news.lower()
    
    # 3. Tokenize.
    news_words = nltk.word_tokenize( news)
    
    return np.array(news_words)

In [0]:
import gensim

google_model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/Shared drives/SheCodes/Datasets/GoogleNews-vectors-negative300.bin.gz', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
from sklearn.preprocessing import StandardScaler

standard_scaler = StandardScaler()

In [0]:
from sklearn.preprocessing import normalize
from functools import reduce

def word2vec_similarity_score(data):
  data['word2vec_cleaned_title'] = data["title"].map(lambda x: cleaning_simple_tokenize(x))
  data['word2vec_cleaned_text'] = data["text"].map(lambda x: cleaning_simple_tokenize(x))

  title_word2vec = data['word2vec_cleaned_title'].map(lambda words: reduce(np.add, [google_model[word] for word in words if word in google_model], [0.]*300))
  text_word2vec = data['word2vec_cleaned_text'].map(lambda words: reduce(np.add, [google_model[word] for word in words if word in google_model], [0.]*300))

  data['word2vec_similarity_title_text'] = similarity_score(data, np.array(title_word2vec), np.array(text_word2vec))



In [0]:
# Word2Vec and Cosine Similarities on Multiclass labeled dataset

word2vec_similarity_score(final_stance)
final_stance.head()


/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


,title,label,text,cleaned_title,cleaned_text,count_title_sentences,count_text_sentences,title_unigram,text_unigram,count_title_unigram,count_text_unigram,unique_count_title_unigram,unique_count_text_unigram,title_bigram,text_bigram,count_title_bigram,count_text_bigram,unique_count_title_bigram,unique_count_text_bigram,title_trigram,text_trigram,count_title_trigram,count_text_trigram,unique_count_title_trigram,unique_count_text_trigram,count_title_unigrams_in_text,count_title_bigrams_in_text,count_title_trigrams_in_text,cleaned_title_text,similarity_title_text,word2vec_cleaned_title,word2vec_cleaned_text,word2vec_similarity_title_text
0,Has Ebola infected Isis militants in Mosul?,discuss,The World Health Organisation is investigating...,ebola infect isi milit mosul,world health organis investig report isi milit...,1,18,"[ebola, infect, isi, milit, mosul]","[world, health, organis, investig, report, isi...",5,260,5,160,"[ebola_infect, infect_isi, isi_milit, milit_mo...","[world_health, health_organis, organis_investi...",4,259,4,231,"[ebola_infect_isi, infect_isi_milit, isi_milit...","[world_health_organis, health_organis_investig...",3,258,3,240,4.0,1.0,0.0,ebola infect isi milit mosul world health orga...,NaN,"[has, ebola, infected, isis, militants, in, mo...","[the, world, health, organisation, is, investi...",0.609588
1,Meteor Leaves 40-Foot Crater Near Managua's Ai...,unrelated,ISIS released a video purportedly showing the ...,meteor leaf foot crater near managua airport,isi releas video purport show execut british a...,1,1,"[meteor, leaf, foot, crater, near, managua, ai...","[isi, releas, video, purport, show, execut, br...",7,11,7,11,"[meteor_leaf, leaf_foot, foot_crater, crater_n...","[isi_releas, releas_video, video_purport, purp...",6,10,6,10,"[meteor_leaf_foot, leaf_foot_crater, foot_crat...","[isi_releas_video, releas_video_purport, video...",5,9,5,9,0.0,0.0,0.0,meteor leaf foot crater near managua airport i...,NaN,"[meteor, leaves, foot, crater, near, managua, ...","[isis, released, a, video, purportedly, showin...",0.249012
2,Mexico prosecutor: Students not in 1st mass gr...,unrelated,After being officially dead for 48 minutes and...,mexico prosecutor student st mass graf,offici dead minut heart restart year old massa...,1,9,"[mexico, prosecutor, student, st, mass, graf]","[offici, dead, minut, heart, restart, year, ol...",6,78,6,69,"[mexico_prosecutor, prosecutor_student, studen...","[offici_dead, dead_minut, minut_heart, heart_r...",5,77,5,77,"[mexico_prosecutor_student, prosecutor_student...","[offici_dead_minut, dead_minut_heart, minut_he...",4,76,4,76,0.0,0.0,0.0,mexico prosecutor student st mass graf offici ...,NaN,"[mexico, prosecutor, students, not, in, st, ma...","[after, being, officially, dead, for, minutes,...",0.528046
3,Letter From Lego To Parents In The '70s Makes ...,unrelated,HBO's subscription streaming service will be c...,letter lego parent make import point gender,hbo subscript stream servic call hbo expect co...,1,16,"[letter, lego, parent, make, import, point, ge...","[hbo, subscript, stream, servic, call, hbo, ex...",7,229,7,137,"[letter_lego, lego_parent, parent_make, make_i...","[hbo_subscript, subscript_stream, stream_servi...",6,228,6,212,"[letter_lego_parent, lego_parent_make, parent_...","[hbo_subscript_stream, subscript_stream_servic...",5,227,5,223,2.0,0.0,0.0,letter lego parent make import point gender hb...,0.0,"[letter, from, lego, to, parents, in, the, s, ...","[hbo, s, subscription, streaming, service, wil...",0.691017
4,Islamic State Militants Have Been Flying 3 Cap...,discuss,By Sylvia Westall\n\nBEIRUT (Reuters) – Iraqi ...,islam state milit fli captur warplan syrian ai...,sylvia westal beirut reuter iraqi pilot join i...,1,18,"[islam, state, milit, fli, captur, warplan, sy...","[sylvia, westal, beirut, reuter, iraqi, pilot,...",10,324,10,191,"[islam_state, state_milit, milit_fli, fli_capt...","[sylvia_westal, westal_beirut, beirut_reuter, ...",9,323,9,299,"[islam_state_milit

In [0]:
# Word2Vec and Cosine Similarities on Binary Class labeled dataset

word2vec_similarity_score(df_final)
df_final.head()

/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


,title,text,label,cleaned_title,cleaned_text,count_title_sentences,count_text_sentences,title_unigram,text_unigram,count_title_unigram,count_text_unigram,unique_count_title_unigram,unique_count_text_unigram,title_bigram,text_bigram,count_title_bigram,count_text_bigram,unique_count_title_bigram,unique_count_text_bigram,title_trigram,text_trigram,count_title_trigram,count_text_trigram,unique_count_title_trigram,unique_count_text_trigram,count_title_unigrams_in_text,count_title_bigrams_in_text,count_title_trigrams_in_text,cleaned_title_text,similarity_title_text,word2vec_cleaned_title,word2vec_cleaned_text,word2vec_similarity_title_text
0,Indian tycoon's extradition hearing told of ab...,LONDON (Reuters) - One of the Chennai Six gr...,0,indian tycoon extradit hear told abus uk chenn...,london reuter one chennai six group ex british...,1,13,"[indian, tycoon, extradit, hear, told, abus, u...","[london, reuter, one, chennai, six, group, ex,...",9,251,9,162,"[indian_tycoon, tycoon_extradit, extradit_hear...","[london_reuter, reuter_one, one_chennai, chenn...",8,250,8,238,"[indian_tycoon_extradit, tycoon_extradit_hear,...","[london_reuter_one, reuter_one_chennai, one_ch...",7,249,7,246,8.0,2.0,0.0,indian tycoon extradit hear told abus uk chenn...,0.134715,"[indian, tycoon, s, extradition, hearing, told...","[london, reuters, one, of, the, chennai, six, ...",0.649405
1,Iran Proves Republicans Are Full Of Crap As U...,Iran is fulfilling their obligations under the...,1,iran prof republican full crap unit nation say...,iran fulfil oblig nuclear deal reach obama adm...,1,20,"[iran, prof, republican, full, crap, unit, nat...","[iran, fulfil, oblig, nuclear, deal, reach, ob...",11,422,11,266,"[iran_prof, prof_republican, republican_full, ...","[iran_fulfil, fulfil_oblig, oblig_nuclear, nuc...",10,421,10,401,"[iran_prof_republican, prof_republican_full, r...","[iran_fulfil_oblig, fulfil_oblig_nuclear, obli...",9,420,9,418,8.0,3.0,0.0,iran prof republican full crap unit nation say...,0.019066,"[iran, proves, republicans, are, full, of, cra...","[iran, is, fulfilling, their, obligations, und...",0.806206
2,Breaking: Dems Keep Progressive Bent In All-I...,Nancy Pelosi has been the leader of the House ...,1,break dem keep progress bent import leadership...,nanci pelosi leader hous democrat long time we...,1,17,"[break, dem, keep, progress, bent, import, lea...","[nanci, pelosi, leader, hous, democrat, long, ...",9,225,9,165,"[break_dem, dem_keep, keep_progress, progress_...","[nanci_pelosi, pelosi_leader, leader_hous, hou...",8,224,8,214,"[break_dem_keep, dem_keep_progress, keep_progr...","[nanci_pelosi_leader, pelosi_leader_hous, lead...",7,223,7,222,4.0,0.0,0.0,break dem keep progress bent import leadership...,0.000000,"[breaking, dems, keep, progressive, bent, in, ...","[nancy, pelosi, has, been, the, leader, of, th...",0.710030
3,nan,Watch this video. Fully documented the heavy c...,1,nan,watch video fulli document heavi connect islam...,1,5,[nan],"[watch, video, fulli, document, heavi, connect...",1,30,1,25,[],"[watch_video, video_fulli, fulli_document, doc...",0,29,0,28,[],"[watch_video_fulli, video_fulli_document, full...",0,28,0,28,0.0,0.0,0.0,nan watch video fulli document heavi connect i...,NaN,[nan],"[watch, this, video, fully, documented, the, h...",0.187610
4,Centrum – The World's Most Popular Multivitami...,"Share on Facebook This can result in fatigue, ...",1,centrum world popular multivitamin useless con...,share facebook result fatigu dizzi short breat...,1,50,"[centrum, world, popular, multivitamin, useles...","[share, facebook, result, fatigu, dizzi, short...",10,434,10,306,"[centrum_world, world_popular, popular_multivi...","[share_facebook, facebook_result, result_fatig...",9,433,9,426,"[centrum_world_popular, world_popular_multivit...","[share_facebook_result, facebook_result_fatigu...",8,432,8,432,9.0,1.0,0.0,centrum world popular multivitamin useless con...,0.224354,"[centrum, the, world, s, most, popular, multiv...","[sh

## LSA Topic Modelling using SVD and Cosine Similarities

Lets now try to find topics from the title and text using SVD. Then find the cosine similarity between the generated title and text topics


In [0]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import vstack

def svd(data, title_tfidf_vectors, text_tfidf_vectors):
  truncated_svd = TruncatedSVD(n_components=100, n_iter=10)
  combined_vectors = vstack([title_tfidf_vectors, text_tfidf_vectors])
  truncated_svd.fit(combined_vectors)

  title_svd = truncated_svd.transform(title_tfidf_vectors)
  print (title_svd.shape)
  text_svd = truncated_svd.transform(text_tfidf_vectors)
  print (text_svd.shape)
  return title_svd, text_svd

def topic_similarity_score(data, title_tfidf_vectors, text_tfidf_vectors):
  title_svd_vectors, text_svd_vectors = svd(data, title_tfidf_vectors, text_tfidf_vectors)
  data['topics_similarity_title_text'] = similarity_score(data, title_svd_vectors, text_svd_vectors)

In [0]:
# LSA Topic Modelling and Cosine Similarities on Multi Class labeled dataset
topic_similarity_score(final_stance, stance_title_tfidf_vectors, stance_text_tfidf_vectors)
final_stance.head()

(754, 100)
(754, 100)


/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


,title,label,text,cleaned_title,cleaned_text,count_title_sentences,count_text_sentences,title_unigram,text_unigram,count_title_unigram,count_text_unigram,unique_count_title_unigram,unique_count_text_unigram,title_bigram,text_bigram,count_title_bigram,count_text_bigram,unique_count_title_bigram,unique_count_text_bigram,title_trigram,text_trigram,count_title_trigram,count_text_trigram,unique_count_title_trigram,unique_count_text_trigram,count_title_unigrams_in_text,count_title_bigrams_in_text,count_title_trigrams_in_text,cleaned_title_text,similarity_title_text,word2vec_cleaned_title,word2vec_cleaned_text,word2vec_similarity_title_text,topics_similarity_title_text
0,Has Ebola infected Isis militants in Mosul?,discuss,The World Health Organisation is investigating...,ebola infect isi milit mosul,world health organis investig report isi milit...,1,18,"[ebola, infect, isi, milit, mosul]","[world, health, organis, investig, report, isi...",5,260,5,160,"[ebola_infect, infect_isi, isi_milit, milit_mo...","[world_health, health_organis, organis_investi...",4,259,4,231,"[ebola_infect_isi, infect_isi_milit, isi_milit...","[world_health_organis, health_organis_investig...",3,258,3,240,4.0,1.0,0.0,ebola infect isi milit mosul world health orga...,NaN,"[has, ebola, infected, isis, militants, in, mo...","[the, world, health, organisation, is, investi...",0.609588,NaN
1,Meteor Leaves 40-Foot Crater Near Managua's Ai...,unrelated,ISIS released a video purportedly showing the ...,meteor leaf foot crater near managua airport,isi releas video purport show execut british a...,1,1,"[meteor, leaf, foot, crater, near, managua, ai...","[isi, releas, video, purport, show, execut, br...",7,11,7,11,"[meteor_leaf, leaf_foot, foot_crater, crater_n...","[isi_releas, releas_video, video_purport, purp...",6,10,6,10,"[meteor_leaf_foot, leaf_foot_crater, foot_crat...","[isi_releas_video, releas_video_purport, video...",5,9,5,9,0.0,0.0,0.0,meteor leaf foot crater near managua airport i...,NaN,"[meteor, leaves, foot, crater, near, managua, ...","[isis, released, a, video, purportedly, showin...",0.249012,NaN
2,Mexico prosecutor: Students not in 1st mass gr...,unrelated,After being officially dead for 48 minutes and...,mexico prosecutor student st mass graf,offici dead minut heart restart year old massa...,1,9,"[mexico, prosecutor, student, st, mass, graf]","[offici, dead, minut, heart, restart, year, ol...",6,78,6,69,"[mexico_prosecutor, prosecutor_student, studen...","[offici_dead, dead_minut, minut_heart, heart_r...",5,77,5,77,"[mexico_prosecutor_student, prosecutor_student...","[offici_dead_minut, dead_minut_heart, minut_he...",4,76,4,76,0.0,0.0,0.0,mexico prosecutor student st mass graf offici ...,NaN,"[mexico, prosecutor, students, not, in, st, ma...","[after, being, officially, dead, for, minutes,...",0.528046,NaN
3,Letter From Lego To Parents In The '70s Makes ...,unrelated,HBO's subscription streaming service will be c...,letter lego parent make import point gender,hbo subscript stream servic call hbo expect co...,1,16,"[letter, lego, parent, make, import, point, ge...","[hbo, subscript, stream, servic, call, hbo, ex...",7,229,7,137,"[letter_lego, lego_parent, parent_make, make_i...","[hbo_subscript, subscript_stream, stream_servi...",6,228,6,212,"[letter_lego_parent, lego_parent_make, parent_...","[hbo_subscript_stream, subscript_stream_servic...",5,227,5,223,2.0,0.0,0.0,letter lego parent make import point gender hb...,0.0,"[letter, from, lego, to, parents, in, the, s, ...","[hbo, s, subscription, streaming, service, wil...",0.691017,-0.028062
4,Islamic State Militants Have Been Flying 3 Cap...,discuss,By Sylvia Westall\n\nBEIRUT (Reuters) – Iraqi ...,islam state milit fli captur warplan syrian ai...,sylvia westal beirut reuter iraqi pilot join i...,1,18,"[islam, state, milit, fli, captur, warplan, sy...","[sylvia, westal, beirut, reuter, iraqi, pilot,...",10,324,10,191,"[islam_state, state_milit, milit_fli, fli_capt...","[sylvia_westal, westal_beirut, 

In [0]:
# LSA Topic Modelling and Cosine Similarities on Binary Class labeled dataset

topic_similarity_score(df_final, df_title_tfidf_vectors, df_text_tfidf_vectors)
df_final.head()

(709, 100)
(709, 100)


/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


,title,text,label,cleaned_title,cleaned_text,count_title_sentences,count_text_sentences,title_unigram,text_unigram,count_title_unigram,count_text_unigram,unique_count_title_unigram,unique_count_text_unigram,title_bigram,text_bigram,count_title_bigram,count_text_bigram,unique_count_title_bigram,unique_count_text_bigram,title_trigram,text_trigram,count_title_trigram,count_text_trigram,unique_count_title_trigram,unique_count_text_trigram,count_title_unigrams_in_text,count_title_bigrams_in_text,count_title_trigrams_in_text,cleaned_title_text,similarity_title_text,word2vec_cleaned_title,word2vec_cleaned_text,word2vec_similarity_title_text,topics_similarity_title_text
0,Indian tycoon's extradition hearing told of ab...,LONDON (Reuters) - One of the Chennai Six gr...,0,indian tycoon extradit hear told abus uk chenn...,london reuter one chennai six group ex british...,1,13,"[indian, tycoon, extradit, hear, told, abus, u...","[london, reuter, one, chennai, six, group, ex,...",9,251,9,162,"[indian_tycoon, tycoon_extradit, extradit_hear...","[london_reuter, reuter_one, one_chennai, chenn...",8,250,8,238,"[indian_tycoon_extradit, tycoon_extradit_hear,...","[london_reuter_one, reuter_one_chennai, one_ch...",7,249,7,246,8.0,2.0,0.0,indian tycoon extradit hear told abus uk chenn...,0.134715,"[indian, tycoon, s, extradition, hearing, told...","[london, reuters, one, of, the, chennai, six, ...",0.649405,0.990605
1,Iran Proves Republicans Are Full Of Crap As U...,Iran is fulfilling their obligations under the...,1,iran prof republican full crap unit nation say...,iran fulfil oblig nuclear deal reach obama adm...,1,20,"[iran, prof, republican, full, crap, unit, nat...","[iran, fulfil, oblig, nuclear, deal, reach, ob...",11,422,11,266,"[iran_prof, prof_republican, republican_full, ...","[iran_fulfil, fulfil_oblig, oblig_nuclear, nuc...",10,421,10,401,"[iran_prof_republican, prof_republican_full, r...","[iran_fulfil_oblig, fulfil_oblig_nuclear, obli...",9,420,9,418,8.0,3.0,0.0,iran prof republican full crap unit nation say...,0.019066,"[iran, proves, republicans, are, full, of, cra...","[iran, is, fulfilling, their, obligations, und...",0.806206,0.271865
2,Breaking: Dems Keep Progressive Bent In All-I...,Nancy Pelosi has been the leader of the House ...,1,break dem keep progress bent import leadership...,nanci pelosi leader hous democrat long time we...,1,17,"[break, dem, keep, progress, bent, import, lea...","[nanci, pelosi, leader, hous, democrat, long, ...",9,225,9,165,"[break_dem, dem_keep, keep_progress, progress_...","[nanci_pelosi, pelosi_leader, leader_hous, hou...",8,224,8,214,"[break_dem_keep, dem_keep_progress, keep_progr...","[nanci_pelosi_leader, pelosi_leader_hous, lead...",7,223,7,222,4.0,0.0,0.0,break dem keep progress bent import leadership...,0.000000,"[breaking, dems, keep, progressive, bent, in, ...","[nancy, pelosi, has, been, the, leader, of, th...",0.710030,0.187350
3,nan,Watch this video. Fully documented the heavy c...,1,nan,watch video fulli document heavi connect islam...,1,5,[nan],"[watch, video, fulli, document, heavi, connect...",1,30,1,25,[],"[watch_video, video_fulli, fulli_document, doc...",0,29,0,28,[],"[watch_video_fulli, video_fulli_document, full...",0,28,0,28,0.0,0.0,0.0,nan watch video fulli document heavi connect i...,NaN,[nan],"[watch, this, video, fully, documented, the, h...",0.187610,NaN
4,Centrum – The World's Most Popular Multivitami...,"Share on Facebook This can result in fatigue, ...",1,centrum world popular multivitamin useless con...,share facebook result fatigu dizzi short breat...,1,50,"[centrum, world, popular, multivitamin, useles...","[share, facebook, result, fatigu, dizzi, short...",10,434,10,306,"[centrum_world, world_popular, popular_multivi...","[share_facebook, facebook_result, result_fatig...",9,433,9,426,"[centrum_world_popular, world_popular_multivit...","[share_facebook_result, facebook_result_fatigu...",8,432,8,432,9.0,1.0,0.0,centrum world popular multivitamin useless con...,0.224

## Sentiment Analysis and Cosine Similaties

Now lets assign polarity sentiment score to the title and text.


In [0]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sentiment_analyzer = SentimentIntensityAnalyzer()

def compute_sentiment(sentences):
    result = []
    for sentence in sentences:
        vs = sentiment_analyzer.polarity_scores(sentence)
        result.append(vs)
    return pd.DataFrame(result).mean()

def title_sentiment(data):
  data['title_sentences'] = data['title'].apply(lambda x: sent_tokenize(x))
  data = pd.concat([data, data['title_sentences'].apply(lambda x: compute_sentiment(x))], axis=1)
  data.rename(columns={'compound':'title_compound', 'neg':'title_neg', 'neu':'title_neu', 'pos':'title_pos'}, inplace=True)
  return data


def text_sentiment(data):
  data['text_sentences'] = data['text'].apply(lambda x: sent_tokenize(x))
  data = pd.concat([data, data['text_sentences'].apply(lambda x: compute_sentiment(x))], axis=1)
  data.rename(columns={'compound':'text_compound', 'neg':'text_neg', 'neu':'text_neu', 'pos':'text_pos'}, inplace=True)
  return data

In [0]:
# Sentiment Analysis on Multi Class labeled dataset

final_stance = title_sentiment(final_stance)
final_stance = text_sentiment(final_stance)
final_stance.head()

,title,label,text,cleaned_title,cleaned_text,count_title_sentences,count_text_sentences,title_unigram,text_unigram,count_title_unigram,count_text_unigram,unique_count_title_unigram,unique_count_text_unigram,title_bigram,text_bigram,count_title_bigram,count_text_bigram,unique_count_title_bigram,unique_count_text_bigram,title_trigram,text_trigram,count_title_trigram,count_text_trigram,unique_count_title_trigram,unique_count_text_trigram,count_title_unigrams_in_text,count_title_bigrams_in_text,count_title_trigrams_in_text,cleaned_title_text,similarity_title_text,word2vec_cleaned_title,word2vec_cleaned_text,word2vec_similarity_title_text,topics_similarity_title_text,title_sentences,title_neg,title_neu,title_pos,title_compound,text_sentences,text_neg,text_neu,text_pos,text_compound
0,Has Ebola infected Isis militants in Mosul?,discuss,The World Health Organisation is investigating...,ebola infect isi milit mosul,world health organis investig report isi milit...,1,18,"[ebola, infect, isi, milit, mosul]","[world, health, organis, investig, report, isi...",5,260,5,160,"[ebola_infect, infect_isi, isi_milit, milit_mo...","[world_health, health_organis, organis_investi...",4,259,4,231,"[ebola_infect_isi, infect_isi_milit, isi_milit...","[world_health_organis, health_organis_investig...",3,258,3,240,4.0,1.0,0.0,ebola infect isi milit mosul world health orga...,NaN,"[has, ebola, infected, isis, militants, in, mo...","[the, world, health, organisation, is, investi...",0.609588,NaN,[Has Ebola infected Isis militants in Mosul?],0.348,0.652,0.000,-0.4939,[The World Health Organisation is investigatin...,0.087056,0.874944,0.038000,-0.100422
1,Meteor Leaves 40-Foot Crater Near Managua's Ai...,unrelated,ISIS released a video purportedly showing the ...,meteor leaf foot crater near managua airport,isi releas video purport show execut british a...,1,1,"[meteor, leaf, foot, crater, near, managua, ai...","[isi, releas, video, purport, show, execut, br...",7,11,7,11,"[meteor_leaf, leaf_foot, foot_crater, crater_n...","[isi_releas, releas_video, video_purport, purp...",6,10,6,10,"[meteor_leaf_foot, leaf_foot_crater, foot_crat...","[isi_releas_video, releas_video_purport, video...",5,9,5,9,0.0,0.0,0.0,meteor leaf foot crater near managua airport i...,NaN,"[meteor, leaves, foot, crater, near, managua, ...","[isis, released, a, video, purportedly, showin...",0.249012,NaN,[Meteor Leaves 40-Foot Crater Near Managua's A...,0.000,1.000,0.000,0.0000,[ISIS released a video purportedly showing the...,0.000000,1.000000,0.000000,0.000000
2,Mexico prosecutor: Students not in 1st mass gr...,unrelated,After being officially dead for 48 minutes and...,mexico prosecutor student st mass graf,offici dead minut heart restart year old massa...,1,9,"[mexico, prosecutor, student, st, mass, graf]","[offici, dead, minut, heart, restart, year, ol...",6,78,6,69,"[mexico_prosecutor, prosecutor_student, studen...","[offici_dead, dead_minut, minut_heart, heart_r...",5,77,5,77,"[mexico_prosecutor_student, prosecutor_student...","[offici_dead_minut, dead_minut_heart, minut_he...",4,76,4,76,0.0,0.0,0.0,mexico prosecutor student st mass graf offici ...,NaN,"[mexico, prosecutor, students, not, in, st, ma...","[after, being, officially, dead, for, minutes,...",0.528046,NaN,[Mexico prosecutor: Students not in 1st mass g...,0.239,0.761,0.000,-0.2960,[After being officially dead for 48 minutes an...,0.111222,0.792667,0.096111,-0.040711
3,Letter From Lego To Parents In The '70s Makes ...,unrelated,HBO's subscription streaming service will be c...,letter lego parent make import point gender,hbo subscript stream servic call hbo expect co...,1,16,"[letter, lego, parent, make, import, point, ge...","[hbo, subscript, stream, servic, call, hbo, ex...",7,229,7,137,"[letter_lego, lego_parent, parent_make, make_i...","[hbo_subscript, subscript_stream, stream_servi...",6,228,6,212,"[letter_lego_parent, lego_parent_make, parent_...","[hbo_subscript_stream, subscript_stream_servic...",5,227,5,223,2.0,0.

In [0]:
# Sentiment Analysis on Binary Class labeled dataset

df_final = title_sentiment(df_final)
df_final = text_sentiment(df_final)
df_final.head()

,title,text,label,cleaned_title,cleaned_text,count_title_sentences,count_text_sentences,title_unigram,text_unigram,count_title_unigram,count_text_unigram,unique_count_title_unigram,unique_count_text_unigram,title_bigram,text_bigram,count_title_bigram,count_text_bigram,unique_count_title_bigram,unique_count_text_bigram,title_trigram,text_trigram,count_title_trigram,count_text_trigram,unique_count_title_trigram,unique_count_text_trigram,count_title_unigrams_in_text,count_title_bigrams_in_text,count_title_trigrams_in_text,cleaned_title_text,similarity_title_text,word2vec_cleaned_title,word2vec_cleaned_text,word2vec_similarity_title_text,topics_similarity_title_text,title_sentences,title_neg,title_neu,title_pos,title_compound,text_sentences,text_neg,text_neu,text_pos,text_compound
0,Indian tycoon's extradition hearing told of ab...,LONDON (Reuters) - One of the Chennai Six gr...,0,indian tycoon extradit hear told abus uk chenn...,london reuter one chennai six group ex british...,1,13,"[indian, tycoon, extradit, hear, told, abus, u...","[london, reuter, one, chennai, six, group, ex,...",9,251,9,162,"[indian_tycoon, tycoon_extradit, extradit_hear...","[london_reuter, reuter_one, one_chennai, chenn...",8,250,8,238,"[indian_tycoon_extradit, tycoon_extradit_hear,...","[london_reuter_one, reuter_one_chennai, one_ch...",7,249,7,246,8.0,2.0,0.0,indian tycoon extradit hear told abus uk chenn...,0.134715,"[indian, tycoon, s, extradition, hearing, told...","[london, reuters, one, of, the, chennai, six, ...",0.649405,0.990605,[Indian tycoon's extradition hearing told of a...,0.296,0.704,0.000,-0.6369,[LONDON (Reuters) - One of the Chennai Six g...,0.091077,0.881462,0.027538,-0.248646
1,Iran Proves Republicans Are Full Of Crap As U...,Iran is fulfilling their obligations under the...,1,iran prof republican full crap unit nation say...,iran fulfil oblig nuclear deal reach obama adm...,1,20,"[iran, prof, republican, full, crap, unit, nat...","[iran, fulfil, oblig, nuclear, deal, reach, ob...",11,422,11,266,"[iran_prof, prof_republican, republican_full, ...","[iran_fulfil, fulfil_oblig, oblig_nuclear, nuc...",10,421,10,401,"[iran_prof_republican, prof_republican_full, r...","[iran_fulfil_oblig, fulfil_oblig_nuclear, obli...",9,420,9,418,8.0,3.0,0.0,iran prof republican full crap unit nation say...,0.019066,"[iran, proves, republicans, are, full, of, cra...","[iran, is, fulfilling, their, obligations, und...",0.806206,0.271865,[ Iran Proves Republicans Are Full Of Crap As ...,0.117,0.586,0.297,0.6124,[Iran is fulfilling their obligations under th...,0.056150,0.804200,0.139650,0.346365
2,Breaking: Dems Keep Progressive Bent In All-I...,Nancy Pelosi has been the leader of the House ...,1,break dem keep progress bent import leadership...,nanci pelosi leader hous democrat long time we...,1,17,"[break, dem, keep, progress, bent, import, lea...","[nanci, pelosi, leader, hous, democrat, long, ...",9,225,9,165,"[break_dem, dem_keep, keep_progress, progress_...","[nanci_pelosi, pelosi_leader, leader_hous, hou...",8,224,8,214,"[break_dem_keep, dem_keep_progress, keep_progr...","[nanci_pelosi_leader, pelosi_leader_hous, lead...",7,223,7,222,4.0,0.0,0.0,break dem keep progress bent import leadership...,0.000000,"[breaking, dems, keep, progressive, bent, in, ...","[nancy, pelosi, has, been, the, leader, of, th...",0.710030,0.187350,[ Breaking: Dems Keep Progressive Bent In All-...,0.000,1.000,0.000,0.0000,[Nancy Pelosi has been the leader of the House...,0.057059,0.823118,0.119824,0.199129
3,nan,Watch this video. Fully documented the heavy c...,1,nan,watch video fulli document heavi connect islam...,1,5,[nan],"[watch, video, fulli, document, heavi, connect...",1,30,1,25,[],"[watch_video, video_fulli, fulli_document, doc...",0,29,0,28,[],"[watch_video_fulli, video_fulli_document, full...",0,28,0,28,0.0,0.0,0.0,nan watch video fulli document heavi connect i...,NaN,[nan],"[watch, this, video, fully, documented, the, h...",0.187610,NaN,[nan],0.000,1.000,0.000,0.00

## Features Distillation for Modelling

Now, lets keep the features which are need for model training and remove all the raw title and text features.



In [0]:
from sklearn import preprocessing

distilled_features_cols = ['label', 'count_title_sentences', 'count_text_sentences', 'count_title_unigram', 'count_text_unigram',
                'unique_count_title_unigram', 'unique_count_text_unigram', 'count_title_bigram', 'count_text_bigram',
                'unique_count_title_bigram', 'unique_count_text_bigram', 'count_title_trigram', 'count_text_trigram',
                'unique_count_title_trigram', 'unique_count_text_trigram', 'count_title_unigrams_in_text', 'count_title_bigrams_in_text',
                'count_title_trigrams_in_text', 'similarity_title_text', 'topics_similarity_title_text', 'word2vec_similarity_title_text',
                'title_neg', 'title_neu', 'title_pos', 'title_compound', 'text_neg', 'text_neu', 'text_pos', 'text_compound']

df_final_multi = final_stance[distilled_features_cols]
le = preprocessing.LabelEncoder()
le.fit(df_final_multi['label'])

df_final_multi['label'] = le.transform(df_final_multi['label'])
# print(le.inverse_transform(df_final_multi['label']))

df_final_multi.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,label,count_title_sentences,count_text_sentences,count_title_unigram,count_text_unigram,unique_count_title_unigram,unique_count_text_unigram,count_title_bigram,count_text_bigram,unique_count_title_bigram,unique_count_text_bigram,count_title_trigram,count_text_trigram,unique_count_title_trigram,unique_count_text_trigram,count_title_unigrams_in_text,count_title_bigrams_in_text,count_title_trigrams_in_text,similarity_title_text,topics_similarity_title_text,word2vec_similarity_title_text,title_neg,title_neu,title_pos,title_compound,text_neg,text_neu,text_pos,text_compound
0,2,1,18,5,260,5,160,4,259,4,231,3,258,3,240,4.0,1.0,0.0,NaN,NaN,0.609588,0.348,0.652,0.000,-0.4939,0.087056,0.874944,0.038000,-0.100422
1,3,1,1,7,11,7,11,6,10,6,10,5,9,5,9,0.0,0.0,0.0,NaN,NaN,0.249012,0.000,1.000,0.000,0.0000,0.000000,1.000000,0.000000,0.000000
2,3,1,9,6,78,6,69,5,77,5,77,4,76,4,76,0.0,0.0,0.0,NaN,NaN,0.528046,0.239,0.761,0.000,-0.2960,0.111222,0.792667,0.096111,-0.040711
3,3,1,16,7,229,7,137,6,228,6,212,5,227,5,223,2.0,0.0,0.0,0.0,-0.028062,0.691017,0.000,0.878,0.122,0.2023,0.019813,0.910812,0.069375,0.157981
4,2,1,18,10,324,10,191,9,323,9,299,8,322,8,315,10.0,1.0,0.0,NaN,NaN,0.780305,0.000,1.000,0.000,0.0000,0.041278,0.950944,0.007778,-0.145089


In [0]:
df_final_binary = df_final[distilled_features_cols]
df_final_binary.head()

,label,count_title_sentences,count_text_sentences,count_title_unigram,count_text_unigram,unique_count_title_unigram,unique_count_text_unigram,count_title_bigram,count_text_bigram,unique_count_title_bigram,unique_count_text_bigram,count_title_trigram,count_text_trigram,unique_count_title_trigram,unique_count_text_trigram,count_title_unigrams_in_text,count_title_bigrams_in_text,count_title_trigrams_in_text,similarity_title_text,topics_similarity_title_text,word2vec_similarity_title_text,title_neg,title_neu,title_pos,title_compound,text_neg,text_neu,text_pos,text_compound
0,0,1,13,9,251,9,162,8,250,8,238,7,249,7,246,8.0,2.0,0.0,0.134715,0.990605,0.649405,0.296,0.704,0.000,-0.6369,0.091077,0.881462,0.027538,-0.248646
1,1,1,20,11,422,11,266,10,421,10,401,9,420,9,418,8.0,3.0,0.0,0.019066,0.271865,0.806206,0.117,0.586,0.297,0.6124,0.056150,0.804200,0.139650,0.346365
2,1,1,17,9,225,9,165,8,224,8,214,7,223,7,222,4.0,0.0,0.0,0.000000,0.187350,0.710030,0.000,1.000,0.000,0.0000,0.057059,0.823118,0.119824,0.199129
3,1,1,5,1,30,1,25,0,29,0,28,0,28,0,28,0.0,0.0,0.0,NaN,NaN,0.187610,0.000,1.000,0.000,0.0000,0.000000,1.000000,0.000000,0.000000
4,1,1,50,10,434,10,306,9,433,9,426,8,432,8,432,9.0,1.0,0.0,0.224354,0.999797,0.783566,0.000,0.795,0.205,0.4754,0.082300,0.804600,0.113120,0.058830


## Model Training

In [0]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, roc_auc_score, f1_score, confusion_matrix, accuracy_score, \
    classification_report, precision_recall_curve, roc_curve, auc, average_precision_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score, train_test_split, cross_val_predict
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")

In [0]:
multi_classifiers = {
    "LogisticRegression": (LogisticRegression(max_iter=10000, solver='lbfgs')),
    "KNearest": (KNeighborsClassifier()),
    "Support Vector Classifier": (SVC()),
    "DecisionTreeClassifier": (DecisionTreeClassifier()),
    "Naive Bayes": (GaussianNB()),
    "Random forests": (RandomForestClassifier()),
    "XGBoost Classifier": (XGBClassifier()),
    "GradientBoostingClassifier": (GradientBoostingClassifier()),
    "MLP Classifier": (MLPClassifier())
}

In [0]:
binary_classifiers = {
    "LogisticRegression": (LogisticRegression(max_iter=10000, solver='lbfgs')),
    "KNearest": (KNeighborsClassifier()),
    "Support Vector Classifier": (SVC()),
    "DecisionTreeClassifier": (DecisionTreeClassifier()),
    "Naive Bayes": (GaussianNB()),
    "Random forests": (RandomForestClassifier()),
    "XGBoost Classifier": (XGBClassifier()),
    "GradientBoostingClassifier": (GradientBoostingClassifier()),
    "MLP Classifier": (MLPClassifier())
}

In [0]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0,1))

def run_all_models(folds, X_final_train, y_final_train, X_final_test, y_final_test, classifiers):
    for classifier_name, classifier in classifiers.items():
        k_fold = StratifiedKFold(n_splits=folds, random_state=100, shuffle=True)
        cross_val_scores = []
        precision_scores = []
        recall_scores = []
        roc_auc_scores = []
        f1_scores = []
        for train_index, test_index in k_fold.split(X_final_train, y_final_train):
            X_train, X_test = pd.DataFrame(data=X_final_train, index=train_index), pd.DataFrame(data=X_final_train, index=test_index)
            y_train, y_test = pd.DataFrame(data=y_final_train, index=train_index), pd.DataFrame(data=y_final_train, index=test_index)
            model = classifier

            model.fit(X_train, y_train.values.ravel())
            scores = cross_val_score(model, X_train, y_train.values.ravel(), cv=5)
            cross_val_scores.append(scores)
            y_pred = model.predict(X_test)
            precision_scores.append(precision_score(y_test, y_pred, average=None))
            recall_scores.append(recall_score(y_test, y_pred, average=None))
            # roc_auc_scores.append(roc_auc_score(y_test, model.predict_proba(X_test), average='macro', multi_class='ovo'))
            f1_scores.append(f1_score(y_test, y_pred, average=None))

        print('============================= {} ============================='.format(classifier_name))
        print('Mean cross validation score: {}'.format(np.array([cross_val_scores]).mean()))
        print('Mean precision score: {}'.format(np.array([precision_scores]).mean()))
        print('Mean Recall score: {}'.format(np.array([recall_scores]).mean()))
        # print('Mean ROC-AUC score: {}'.format(np.array([roc_auc_scores]).mean()))
        print('Mean F1 score: {}'.format(np.array([f1_scores]).mean()))
        print('******* Real test dataset metrics *******')
        y_final_pred = model.predict((X_final_test))
        print('Accuracy score for the real test set:\n', accuracy_score(y_final_test, y_final_pred))
        print('confusion matrix for the real test set:\n', confusion_matrix(y_final_test, y_final_pred))
        print('Classification report for the real test set:\n', classification_report(y_final_test, y_final_pred))
        # if classifier_name != 'Support Vector Classifier':
        #     y_final_pred_prob = model.predict_proba(X_test)
        #     plot_auc_roc_curve(y_test, y_final_pred_prob[:, 1], classifier_name)
        #     plot_precision_recall_curve(y_test, y_final_pred_prob[:, 1], classifier_name)

## AUC-ROC Curves

In [0]:
def plot_auc_roc_curve(y_test, y_pred, name):
    fpr, tpr, thresholds = roc_curve(y_test.to_numpy(), y_pred)
    roc_auc = auc(fpr, tpr)

    # Plot ROC
    plt.title('ROC for {}'.format(name))
    plt.plot(fpr, tpr, 'b', label='AUC = %0.2f' % roc_auc)
    plt.legend(loc='lower right')
    plt.plot([0, 1], [0, 1], 'r--')
    plt.xlim([-0.1, 1.0])
    plt.ylim([-0.1, 1.01])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

## Precision Recall Curve

In [0]:
def plot_precision_recall_curve(y_test, y_pred_prob, name):
    # Generate precision recall curve values: precision, recall, thresholds
    precision, recall, thresholds = precision_recall_curve(y_test.to_numpy(), y_pred_prob)

    # Plot Precision Recall curve
    plt.plot(recall, precision)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    avg_precision_score = average_precision_score(y_test, y_pred_prob)
    plt.title('PRC for {} - avg precision score: {}'.format(name, str(avg_precision_score)))
    plt.show()

*italicized text*### Model training on Multiclass classification dataset

In [0]:
df_final_multi.fillna(0, inplace=True)

y = df_final_multi['label']
X = df_final_multi.drop(['label'], axis=1)

X.reset_index(drop=True, inplace=True)
y.reset_index(drop=True, inplace=True)

X_final_train, X_final_test, y_final_train, y_final_test = train_test_split(X, y, test_size=0.20, random_state=100, stratify=y)
X_final_train.reset_index(drop=True, inplace=True)
X_final_test.reset_index(drop=True, inplace=True)
y_final_train.reset_index(drop=True, inplace=True)
y_final_test.reset_index(drop=True, inplace=True)

run_all_models(5, X_final_train, y_final_train, X_final_test, y_final_test, multi_classifiers)

============================= LogisticRegression =============================
Mean cross validation score: 0.8664948453608248
Mean precision score: 0.43838629534553314
Mean Recall score: 0.4608374384236453
Mean F1 score: 0.4413434060644761
******* Real test dataset metrics *******
Accuracy score for the real test set:
 0.8807947019867549
confusion matrix for the real test set:
 [[  0   0  10   1]
 [  2   0   1   0]
 [  0   0  24   3]
 [  0   0   1 109]]
Classification report for the real test set:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        11
           1       0.00      0.00      0.00         3
           2       0.67      0.89      0.76        27
           3       0.96      0.99      0.98       110

    accuracy                           0.88       151
   macro avg       0.41      0.47      0.43       151
weighted avg       0.82      0.88      0.85       151

============================= KNearest =====================

### Multiclass Classification Results

As we can see above, Random Forest and XgBoost models perfomed well for multi classfication.

### Model training on Binary classification dataset

In [0]:
df_final_binary.fillna(0, inplace=True)

y = df_final_binary['label']
X = df_final_binary.drop(['label'], axis=1)

X.reset_index(drop=True, inplace=True)
y.reset_index(drop=True, inplace=True)

X_final_train, X_final_test, y_final_train, y_final_test = train_test_split(X, y, test_size=0.20, random_state=100, stratify=y)
X_final_train.reset_index(drop=True, inplace=True)
X_final_test.reset_index(drop=True, inplace=True)
y_final_train.reset_index(drop=True, inplace=True)
y_final_test.reset_index(drop=True, inplace=True)

run_all_models(5, X_final_train, y_final_train, X_final_test, y_final_test, binary_classifiers)

============================= LogisticRegression =============================
Mean cross validation score: 0.7191208791208791
Mean precision score: 0.7238496439257158
Mean Recall score: 0.721692161438078
Mean F1 score: 0.7208227998356921
******* Real test dataset metrics *******
Accuracy score for the real test set:
 0.6690140845070423
confusion matrix for the real test set:
 [[50 20]
 [27 45]]
Classification report for the real test set:
               precision    recall  f1-score   support

           0       0.65      0.71      0.68        70
           1       0.69      0.62      0.66        72

    accuracy                           0.67       142
   macro avg       0.67      0.67      0.67       142
weighted avg       0.67      0.67      0.67       142

============================= KNearest =============================
Mean cross validation score: 0.6097680097680098
Mean precision score: 0.6283626907549988
Mean Recall score: 0.618265491314493
Mean F1 score: 0.6104931114729188

### Binary Classification Results

As we can see above, Gradient Boost Classifierperfomed well for binary classfication.

## Prediction and Testing
Lets create methods that take title and text, generate all features required for the model and predict the multi class label.

In [0]:
def get_distilled_dataset(title, text):
  data = {'title': [title], 'text': [text]}
  df_test = pd.DataFrame(data)

	# Add the cleaned columns
  df_test['cleaned_title'] = df_test["title"].map(lambda x: cleaning(x))
  df_test['cleaned_text'] = df_test["text"].map(lambda x: cleaning(x))

	# Add sentence count columns
  count_sentences(df_test)

	# Add ngram features
  generate_ngram_features(df_test)

	# Add common ngram counts
  common_ngrams_in_text(df_test)

	# Add TF_IDF and similarity scores
  title_tfidf_vectors, text_tfidf_vectors = tf_idf_similarity_score(df_test)

	# Add word2vec similarity scores
  word2vec_similarity_score(df_test)

	# Add Topic Similarity score
  topic_similarity_score(df_test, title_tfidf_vectors, text_tfidf_vectors)

	# Add Sentiment scores
  df_test = title_sentiment(df_test)
  df_test = text_sentiment(df_test)

	# Distill features
  X_cols = [x for i,x in enumerate(distilled_features_cols) if x!='label']
  return df_test[X_cols]

## Pickle the models

Pickling the trained models so that they can be loaded into the the notebook that has other feature analysis


In [0]:
import pickle

multi_class_model = multi_classifiers['Random forests']
binary_class_model = binary_classifiers['GradientBoostingClassifier']


multi_class_model_path = '/content/drive/Shared drives/SheCodes/MLSpring2020/shecodes_employee_attrition/Alternus Vera Sprint 4/Models/multi_class_model.pickle'
binary_class_model_path = '/content/drive/Shared drives/SheCodes/MLSpring2020/shecodes_employee_attrition/Alternus Vera Sprint 4/Models/binary_class_model.pickle'


pickle.dump(multi_class_model, open(multi_class_model_path, 'wb'))
pickle.dump(binary_class_model, open(binary_class_model_path, 'wb'))

In [0]:
multi_class_model = pickle.load(open(multi_class_model_path, "rb"))
binary_class_model = pickle.load(open(binary_class_model_path, "rb"))

In [0]:
def predict_multiclass(title, text):
  df_test = get_distilled_dataset(title, text)
  return multi_classifiers['Random forests'].predict(df_test), multi_classifiers['Random forests'].predict_proba(df_test)

In [0]:
def predict_binaryclass(title, text):
  df_test = get_distilled_dataset(title, text)
  return binary_classifiers['GradientBoostingClassifier'].predict(df_test), binary_classifiers['GradientBoostingClassifier'].predict_proba(df_test)

My Polynomial equation gives 60%, 40% weights to the accuracies respectively given by multi label classification predicted by Random Forest and binary classification predicted by Gradient Boost Classifer.

For generating the fakeness score in the range of 0 to 1, where 1 denotes the news is absolutely fake, I am considering `Agrees` and `Discusses` categories from the Fake News Dataset as True classification. Remaining labels `Disagrees` and `Unrelated` are considered fake. 

If prediction agrees, probability of being True is higher as many features might have correlated. So I am giving weight of 40% to `Agrees`, 20% to `Discusses`, 30% to `Disagrees` and 10% to `Unrelated`.

So My polynomial would be something like below

In [0]:
def Shecodes_getTitleVsBodyScore(title, text):
  prediction_multi, prediction_multi_prob = predict_multiclass(title, text)
  print (prediction_multi)
  print ("Multi label predicted classification: {}".format(le.inverse_transform(prediction_multi)[0]))
  print ("Multi label prediction probabilities {} : {}".format(le.inverse_transform([0,1,2,3]), prediction_multi_prob))

  prediction_binary, prediction_binary_prob = predict_binaryclass(title, text)
  print (prediction_binary)
  print ("Binary predicted classification: {}".format("Fake" if prediction_binary[0] == 1 else "True"))
  print ("Binary label prediction probabilities ['True', 'Fake']: {}".format(prediction_binary_prob))
  print (prediction_multi)

  multiclass_truth_score = (prediction_multi_prob[0][0] * 0.6 + prediction_multi_prob[0][2] + 0.4)
  multiclass_fake_score = (prediction_multi_prob[0][1] * 0.6 + prediction_multi_prob[0][3] + 0.4)

  binaryclass_truth_score = (prediction_binary_prob[0][0] * 0.5)
  binaryclass_fake_score = (prediction_binary_prob[0][1] * 0.5)

  overall_truth_score = (0.6 * multiclass_truth_score) + (0.4 * binaryclass_truth_score)
  overall_fakeness_score = (0.6 * multiclass_fake_score) + (0.4 * binaryclass_fake_score)

  if (overall_truth_score > overall_fakeness_score):
    if (overall_truth_score > 1):
      overall_truth_score = 1
    print ("Overall Fakeness Score: {}", 1 - overall_truth_score)
    return 1 - overall_truth_score
  else:
    if (overall_fakeness_score > 1):
      overall_fakeness_score = 1
    print ("Overall Fakeness Score: {}", overall_fakeness_score)
    return overall_fakeness_score

In [0]:
score = Shecodes_getTitleVsBodyScore (
  "A Russian Guy Says His Justin Bieber Ringtone Saved Him From A Bear Attack", 
  "A bereaved Afghan mother took revenge on the Taliban after watching them kill her son in an ambush. Reza Gul killed 25 Taliban fighters and injured five others in a seven-hour gunbattle in Farah province. Gul, who was joined by her daughter and daughter in-law, engaged the Taliban using AK-47s and grenades, despitenever before having used a weapon. The embattled mother told Tolo news, a 24-hour Afghan news broadcaster, she was awakened by shots early Tuesday. After seeing that her son had been killed, Gul and the other two women fought back. “I couldn't stop myself and picked up a weapon,” Gul told Tolo News. “I went to the check post and began shooting back.” Seema, her daughter-in-law, added: “The fighting was intensified when we reached the battlefield along with light and heavy weapons. We were committed to fight until the last bullet.” Gul said that the battlefield was covered in Talib fighters after the deadly exchange ended. While the Taliban have not publicly commented on the incident, the Afghan government labeled it a symbol of a public uprising against the Taliban. Taliban and other groups have regained large swathes of the country as U.S. and NATO forces slowly pull out troops after 14 years of war. The Taliban have targeted government and foreign infrastructure as the group attempts to claw back power it lost in 2001. While the Taliban have made key gains in rural regions, members continue to employ suicide bomber tactics in well protected towns and cities. Earlier this week, 50 people were killed after a suicide bomber detonated a vest during a volleyball competition in Yahyakahil, Paktika province. That particular attack prompted President Ashraf Ghani to order a complete overview of the country’s defense forces and to rethink the ban on nighttime raids, which were outlawed by his predecessor, Hamid Karzai."
)
print (score)

score = Shecodes_getTitleVsBodyScore (
  "Donald Trump Sends Out Embarrassing New Year‚Äôs Eve Message; This is Disturbing", 
  "Donald Trump just couldn t wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  The former reality show star had just one job to do and he couldn t do it. As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year,  President Angry Pants tweeted.  2018 will be a great year for America! As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year. 2018 will be a great year for America!  Donald J. Trump (@realDonaldTrump) December 31, 2017Trump s tweet went down about as welll as you d expect.What kind of president sends a New Year s greeting like this despicable, petty, infantile gibberish? Only Trump! His lack of decency won t even allow him to rise above the gutter long enough to wish the American citizens a happy new year!  Bishop Talbert Swan (@TalbertSwan) December 31, 2017no one likes you  Calvin (@calvinstowell) December 31, 2017Your impeachment would make 2018 a great year for America, but I ll also accept regaining control of Congress.  Miranda Yaver (@mirandayaver) December 31, 2017Do you hear yourself talk? When you have to include that many people that hate you you have to wonder? Why do the they all hate me?  Alan Sandoval (@AlanSandoval13) December 31, 2017Who uses the word Haters in a New Years wish??  Marlene (@marlene399) December 31, 2017You can t just say happy new year?  Koren pollitt (@Korencarpenter) December 31, 2017Here s Trump s New Year s Eve tweet from 2016.Happy New Year to all, including to my many enemies and those who have fought me and lost so badly they just don t know what to do. Love!  Donald J. Trump (@realDonaldTrump) December 31, 2016This is nothing new for Trump. He s been doing this for years.Trump has directed messages to his  enemies  and  haters  for New Year s, Easter, Thanksgiving, and the anniversary of 9/11. pic.twitter.com/4FPAe2KypA  Daniel Dale (@ddale8) December 31, 2017Trump s holiday tweets are clearly not presidential.How long did he work at Hallmark before becoming President?  Steven Goodine (@SGoodine) December 31, 2017He s always been like this . . . the only difference is that in the last few years, his filter has been breaking down.  Roy Schulze (@thbthttt) December 31, 2017Who, apart from a teenager uses the term haters?  Wendy (@WendyWhistles) December 31, 2017he s a fucking 5 year old  Who Knows (@rainyday80) December 31, 2017So, to all the people who voted for this a hole thinking he would change once he got into power, you were wrong! 70-year-old men don t change and now he s a year older.Photo by Andrew Burton/Getty Images."
)
print (score)

score = Shecodes_getTitleVsBodyScore (
  "Bad News For Trump ‚Äî Mitch McConnell Says No To Repealing Obamacare In 2018", 
  "Republicans have had seven years to come up with a viable replacement for Obamacare but they failed miserably. After taking a victory lap for gifting the wealthy with a tax break on Wednesday, Donald Trump looked at the cameras and said,  We have essentially repealed Obamacare and we will come up with something that will be much better. Obamacare has been repealed in this bill,  he added. Well, like most things Trump says, that s just not true. But, if the former reality show star could have done that in order to eradicate former President Obama s signature legislation, he would have and without offering an alternative.Senate Majority Leader Mitch McConnell told NPR that  This has not been a very bipartisan year. I hope in the new year, we re going to pivot here and become more cooperative. An Obamacare repeal in 2018 is DOA. Well, we obviously were unable to completely repeal and replace with a 52-48 Senate,  the Kentucky Republican said.  We ll have to take a look at what that looks like with a 51-49 Senate. But I think we ll probably move on to other issues. NPR reports:McConnell hopes to focus instead on stabilizing the insurance marketplaces to keep premiums from skyrocketing in the early months of 2018, a promise he made to moderate Republican Sen. Susan Collins of Maine to get her support for the tax bill.On top of that McConnell broke with House Speaker Paul Ryan, R-Wis., on the approach to paring back spending on programs like Medicaid and food stamps. McConnell told NPR he is  not interested  in using Senate budget rules to allow Republicans to cut entitlements without consultation with Democrats. I think entitlement changes, to be sustained, almost always have to be bipartisan,  McConnell said.  The House may have a different agenda. If our Democratic friends in the Senate want to join us to tackle any kind of entitlement reform. I d be happy to take a look at it. This is coming from Mitch McConnell. He knows Donald Trump is destroying the GOP. It doesn t matter, Sen. McConnell. We still recall him saying that his  number one priority is making sure president Obama s a one-term president. Well, we re hoping that Trump doesn t last a full term. Funny how that works.Photo by Chip Somodevilla/Getty Images"
)
print (score)

score = Shecodes_getTitleVsBodyScore (
  " Judge Who Barred A Mom From Seeing Her Baby For A Year Over Unpaid Fees Resigns", 
  "With all the recent talk of sentencing reform and unfair bail across America, sometimes what it takes is an outrageous case in a small town somewhere to light a fire under activists to demand a change. And even though the resignation of a judge in Pearl, Mississippi, a suburb of the capital with a meager 25,000 residents, isn t really anyone s idea of a revolution, it probably means a lot to the mother at the center of the case.Judge John Shirley of the now-dissolved Pearl Youth Court decided back in August of 2016 that some unpaid court fees were enough to separate  Mother A,  a resident of nearby Jackson, from her baby for a period of 14 months. The unidentified mom and a friend were driving through Pearl one day looking for work when they were pulled over by a police officer who discovered that both women had misdemeanor warrants for minor offenses.Honestly, the story begins with that officer, who is also unidentified in the report from the Clarion-Ledger, the news outlet that first reported the judge s resignation. So much begins with a cop s interaction with a young person   will they detain them? Will they give them a warning? Initial contact with law enforcement often determines the course of a young person s life, and most police are totally aware of that fact. Mother A  had her baby with her that day, and although the woman s grandmother came to the scene immediately to collect the baby, that officer decided to change the young woman s life by requiring her to bring the child before the Pearl Youth Court.If it seems like I m being too hard on the cop, let me assure you, I ve saved most of my hate for Judge Shirley. How an arbiter of justice thinks ordering a young mother to stay away from her newborn baby is anything approaching justice is bewildering to me. But that s just what Judge Shirley did, remanding the infant to the custody of the grandmother and imposing a goddamn NO CONTACT ORDER between Mother A and her baby until the poor (as in, without money) young woman could pay off her court fees.Not only could she not see her child, she couldn t see her grandmother while the baby was present. It was just to be cruel. That s the only explanation.Cliff Johnson, the Director of the Roderick and Solange MacArthur Justice Center at the University of Mississippi School of Law, agreed. As a civil rights lawyer in Mississippi, I am no stranger to injustice, but for a judge to prohibit an impoverished mother from having any contact with her baby until monetary payments are made is shocking and repugnant. Such orders are tantamount to judicial kidnapping. Between two people   one cop and one judge   a young woman s life is forever changed, her idea of justice forever skewed, her trust in authority forever hobbled by the fear of a cruelty that exists only for its own sake.Featured image via judgejohnshirley.com"
)
print (score)

(1, 328)
(1, 328)
(1, 2)
(1, 2)
[3]
Multi label predicted classification: unrelated
Multi label prediction probabilities ['agree' 'disagree' 'discuss' 'unrelated'] : [[0.01 0.01 0.06 0.92]]
(1, 328)
(1, 328)
(1, 2)
(1, 2)
[1]
Binary predicted classification: Fake
Binary label prediction probabilities ['True', 'Fake']: [[0.39808741 0.60191259]]
[3]
Overall Fakeness Score: {} 0.9159825177842689
0.9159825177842689
(1, 375)
(1, 375)
(1, 2)
(1, 2)
[2]
Multi label predicted classification: discuss
Multi label prediction probabilities ['agree' 'disagree' 'discuss' 'unrelated'] : [[0.37 0.01 0.47 0.15]]
(1, 375)
(1, 375)
(1, 2)
(1, 2)
[1]
Binary predicted classification: Fake
Binary label prediction probabilities ['True', 'Fake']: [[0.12787245 0.87212755]]
[2]
Overall Fakeness Score: {} 0.3192255101875816
0.3192255101875816
(1, 366)
(1, 366)
(1, 2)
(1, 2)
[2]
Multi label predicted classification: discuss
Multi label prediction probabilities ['agree' 'disagree' 'discuss' 'unrelated'] : [[0.35 0